# Notebook I - Climate Regime
<hr>
This module performs climate data analysis and compiling general agro-climatic indicators. These general agro-climatic indicators summarize climatic profiles in the study area for each grid. The key input data for this module is the climatic data, and the geographical and terrain data.

Prepared by Geoinformatics Center, AIT
<hr>


In [ ]:
import psutil
# psutil.virtual_memory()

psutil.virtual_memory().free/1E9

Now, we will import the specific Python packages we need for PyAEZ.

In [1]:
'''import supporting libraries'''
import numpy as np
import matplotlib.pyplot as plt
# import numpy as np
import os
try:
    from osgeo import gdal
except:
    import gdal
import sys

from time import time as timer

import xarray as xr
import rioxarray as rio

import dask.array as da
import dask

import psutil
from dask.distributed import Client,LocalCluster
# import graphviz

In [2]:
# branch version tag
revname='v21pv'

# # HPC Orion
# # Replace with path to your PyAEZ folder under your username
# work_dir = '/work/hpc/users/kerrie/UN_FAO/repos/PyAEZ/pyaez2.1_vectorize/'
# # Replace with whatever location you want to output data under your username
# out_path = '/work/hpc/users/kerrie/UN_FAO/pyaez_results/china_8110/'+revname+'/' 
# # these are the same for everyone on HPC Orion
# data_dir = '/work/hpc/datasets/un_fao/pyaez/china_8110/daily/npy/'
# maskfile = '/work/hpc/datasets/un_fao/pyaez/china_static/netcdf/mask.nc'
# elevfile = '/work/hpc/datasets/un_fao/pyaez/china_static/tif/elev.tif'


# # Kerrie desktop
# work_dir = 'K:/projects/unfao/pyaez_gaez/repos/PyAEZ_kerrie/PyAEZ/pyaez2.1_parvec/' # path to your PyAEZ repo
# out_path = work_dir+'NB1outputs/' # path for saving output data
# data_dir = 'C://Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_china_03272023/npy/' # path to your data
# maskfile = 'C://Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_china_03272023/tif/mask.tif'# subset for no antarctica, 1800 lats
# elevfile = 'C://Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_china_03272023/tif/elev.tif'

# work_dir = 'K:/projects/unfao/pyaez_gaez/repos/PyAEZ_kerrie/PyAEZ/pyaez2.1_parvec/' # path to your PyAEZ repo
# out_path = work_dir+'NB1outputs/' # path for saving output data
# data_dir = 'C://Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_global_NOTPRODUCTION/npy/' # path to your data
# maskfile = 'C://Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_global_NOTPRODUCTION/tif/mask_2268708_5m.tif'# subset for no antarctica, 1800 lats
# elevfile = 'C://Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_global_NOTPRODUCTION/tif/Elevation_2268708_5m.tif'


# # Kerrie laptop china
# work_dir = 'C://Users/kerrie/Documents/01_LocalCode/repos/PyAEZ/pyaez2.1_parvec/' # path to your PyAEZ repo
# out_path = work_dir+'NB1outputs/' # path for saving output data
# # # data_dir = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/npy/' # path to your data
# # # maskfile = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/tif/mask.tif'# subset for no antarctica, 1800 lats
# # # elevfile = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/china/tif/elev.tif'
# data_dir = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/5GB/' # path to your data
# maskfile = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/5GB/mask.tif'# subset for no antarctica, 1800 lats
# elevfile = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/5GB/elev.tif'

# # Kerrie laptop global
work_dir = 'C://Users/kerrie/Documents/01_LocalCode/repos/PyAEZ/pyaez2.1_parvec/' # path to your PyAEZ repo
out_path = work_dir+'NB1outputs/' # path for saving output data
data_dir = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/global_NOTPRODUCTION/npy/' # path to your data
maskfile = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/global_NOTPRODUCTION/tif/mask_2268708_5m.tif'# subset for no antarctica, 1800 lats
elevfile = 'C://Users/kerrie/Documents/02_LocalData/pyAEZ_input_data/global_NOTPRODUCTION/tif/Elevation_2268708_5m.tif'

# Check whether the specified path exists or not
isExist = os.path.exists(out_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(out_path)
   print("The new directory is created!")

dir_toplev='C://Users/kerrie/Documents/01_LocalCode/'
daskpath=os.path.join(dir_toplev, "dask-worker-space-can-be-deleted")

isExist = os.path.exists(daskpath)
if not isExist:
   os.mkdir(daskpath)
   print("dask worker directory created")

In [3]:
# %load_ext autoreload
# %autoreload 2
# Import Module 1 and initate Class intance
# from pyaez import ClimateRegime
# clim_reg = ClimateRegime.ClimateRegime()

# # Importing UtilitiesCalc
# from pyaez import UtilitiesCalc
# obj_util = UtilitiesCalc.UtilitiesCalc()
sys.path.append(work_dir)
# import ClimateRegime_v21pv as ClimateRegime
# import ClimateRegime_test as ClimateRegime
import ClimateRegime_loopchunks as ClimateRegime
clim_reg = ClimateRegime.ClimateRegime()

# import UtilitiesCalc_v21pv as UtilitiesCalc
import UtilitiesCalc_test as UtilitiesCalc
obj_utilities=UtilitiesCalc.UtilitiesCalc()


# import UtilitiesCalc_v21pv as UtilitiesCalc
import LGPCalc_loopchunks as LGPCalc


In [4]:
# # cluster=LocalCluster(n_workers=1, threads_per_worker=20)#, memory_limit='10GB')
# # client=Client(cluster)
# client=Client()
# client

In [5]:
# expect 2 seconds run time for china
# expect s run time for global

start=timer()

max_temp = da.from_npy_stack(data_dir+'Tmax-2m365/').astype('float32')  # maximum temperature
min_temp = da.from_npy_stack(data_dir+'Tmin-2m365/').astype('float32')  # minimum temperature
precipitation = da.from_npy_stack(data_dir+'Precip365/').astype('float32')  # precipitation
rel_humidity = da.from_npy_stack(data_dir+'Rhum365/').astype('float32')  # relative humidity
wind_speed = da.from_npy_stack(data_dir+'Wind-2m365/').astype('float32') # wind speed measured at two meters
short_rad = da.from_npy_stack(data_dir+'Srad365/').astype('float32')  # shortwave radiation
mask=da.from_array(gdal.Open(maskfile).ReadAsArray())
elevation=da.from_array(gdal.Open(elevfile).ReadAsArray())

print(min_temp.shape,mask.shape)

task_time=timer()-start
task_time

(1800, 4320, 365) (1800, 4320)


c:\Users\kerrie\.conda\envs\pyaez_dask\Lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


0.07602405548095703

In [6]:
# test=np.nan_to_num(short_rad)
# test[test<0]=100
# test.max().compute()

In [7]:
(max_temp.nbytes+min_temp.nbytes+precipitation.nbytes+rel_humidity.nbytes+wind_speed.nbytes+short_rad.nbytes+mask.nbytes+elevation.nbytes)/1E9

68.179968

In [8]:
# Define the Area-Of-Interest's geographical extents

# if lat_min/lat_max values defined below are located at pixel center --> set lat_centers to True 
# if they are located at the exterior pixel edge --> set lat_centers to False
lat_centers=True 

# provide min and max latitudes (either set manually or read from a data file)
# lat_min = 18.04167
# lat_max = 53.625
lats=rio.open_rasterio(maskfile)['y'].data   # get array of latitudes from maskfile
lat_min = np.trunc(lats.min()*100000)/100000 # min lat value at pixel center, limit precision to 5 decimal places
lat_max = np.trunc(lats.max()*100000)/100000 # max lat value at pixel center, limit precision 5 decimal places

# mask information
mask_value = 0  # pixel value in admin_mask to exclude from the analysis
mask_path=maskfile

# time resolution information
daily = True #Type of climate data = True: daily, False: monthly
parallel=True#False#True

### Loading the imported data into the Object Class ('*clim_reg*' Class)

In [9]:
# expect >1s run time for china
start=timer()

clim_reg.setParallel(max_temp,parallel)#,nchunks=288)#,nchunks=864)

task_time=timer()-start
task_time

computed optimized nchunks,nlons 140 31


0.002989053726196289

In [10]:
# expect >1s run time for china
start=timer()

clim_reg.setStudyAreaMask(mask, mask_value)

task_time=timer()-start
task_time

0.001993894577026367

In [11]:
# expect >1s run time for china
start=timer()

clim_reg.setLocationTerrainData(lat_min, lat_max, lat_centers, elevation) #KLG

task_time=timer()-start
task_time


0.32726407051086426

In [12]:
clim_reg.nchunks, clim_reg.chunk3D, clim_reg.chunksize3D_MB

(140, (-1, 31, -1), 81.468)

In [13]:
print(psutil.virtual_memory().free/1E9)

26.840612864


In [14]:
# parallel=True; for global expect ~8.5min (laptop), ~11.5min (desktop) run time
# parallel=True; for china expect x (laptop), ~24s (desktop) run time
# parallel=False; for china
# for parallel this is computing pet_daily and a few 2D monthly means and saving in memory as numpy arrays
# everything else remains a dask array to be loaded and computed later when needed
start=timer()

clim_reg.setDailyClimateData(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

task_time=timer()-start
task_time

in ClimateRegime, computing pet_daily in parallel
in ClimateRegime, agg daily to meanT_monthly_sealevel in parallel
in ClimateRegime, computing P_by_PET in parallel
in ClimateRegime, computing meanT_monthly, totalPrec_monthly
in ClimateRegime, computing annual_Tmean
in ClimateRegime, computing annual_accPrec


429.5263307094574

In [ ]:
# start=timer()

# tclimate = clim_reg.getThermalClimate()

# task_time=timer()-start
# task_time

In [15]:
lgpt5 = clim_reg.getThermalLGP5()
print(lgpt5.dtype)
print(psutil.virtual_memory().free/1E9)

in ClimateRegime, computing lgpt5 in parallel
float32
25.689513984


In [16]:
start=timer()

# lgp = clim_reg.getLGP(Sa=100., D=1.)
lgp = clim_reg.getLGP(Sa=100., D=1.)

task_time=timer()-start
task_time

time spent on prepping vars 84.10980415344238
loop 0
{'chunk_size': (1800, 72, 365), 'complete day loop': 7.547747611999512, 'avg daily compute': 0.010507117232231244, 'avg daily agg': 0.010099114457221881, 'stack': 0.3182706832885742, 'ETA where': 0.06881833076477051, 'append': 0.08271646499633789, 'xx,yy': 0.6072328090667725, 'lgp_whole': 0.04783987998962402, 'lgp_tot': 0.10066604614257812}
loop 1
{'chunk_size': (1800, 72, 365), 'complete day loop': 7.128700017929077, 'avg daily compute': 0.00958121312807684, 'avg daily agg': 0.00990078285948871, 'stack': 0.3040282726287842, 'ETA where': 0.06876850128173828, 'append': 0.07276320457458496, 'xx,yy': 0.6382825374603271, 'lgp_whole': 0.04987192153930664, 'lgp_tot': 0.0997624397277832}
loop 2
{'chunk_size': (1800, 72, 365), 'complete day loop': 7.124741792678833, 'avg daily compute': 0.009497321141909246, 'avg daily agg': 0.009959672248526796, 'stack': 0.3389298915863037, 'ETA where': 0.06777095794677734, 'append': 0.07276153564453125, 'x

767.4514420032501

In [ ]:
plt.imshow(lgp,interpolation='none')
# plt.imshow(results[0],interpolation='none')
plt.colorbar()

In [ ]:
def psh(ng, et0):
    psh0=da.where(ng==0,0.5,0.3+(ng-1)*.05)
    psh = psh0 + .04 * (5.-et0)
    psh=da.where(psh<0.1,0.1,psh)  
    psh=da.where(psh>0.8,0.8,psh)

    return psh.astype('float32')

In [ ]:
start=timer()
Sa=100.
D=1.
Txsnm = 0. 

# bigchunk2D=(-1,50)
# bigchunk3D=(-1,50,-1)
# nchunks=32
# bigchunk2D=(-1,198)
# bigchunk3D=(-1,198,-1)
# nchunks=8
bigchunk2D=(-1,80)
bigchunk3D=(-1,80,-1)
nchunks=54#20
# bigchunk2D=(-1,99)
# bigchunk3D=(-1,99,-1)
# nchunks=16

print('build task graph for istart0,istart1,p')
# lgpt5=da.from_array(clim_reg.lgpt5,chunks=clim_reg.chunk2D)
# istart0,istart1=LGPCalc.rainPeak(clim_reg.meanT_daily,lgpt5)
# ng=da.zeros(clim_reg.pet_daily.shape,chunks=clim_reg.chunk3D,dtype='float32')
# pet=da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D)
lgpt5=da.from_array(clim_reg.lgpt5,chunks=bigchunk2D)
istart0,istart1=LGPCalc.rainPeak(clim_reg.meanT_daily.rechunk(chunks=bigchunk3D),lgpt5)
ng=da.zeros(clim_reg.pet_daily.shape,chunks=bigchunk3D,dtype='float32')
pet=da.from_array(clim_reg.pet_daily,chunks=bigchunk3D)

print('compute eta_class')
# lgpt5_3D=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis].astype('float16'),(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
# mask_3D=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
# Tmean=clim_reg.meanT_daily.astype('float16').to_delayed().ravel()
# Tmax=clim_reg.maxT_daily.astype('float16').to_delayed().ravel()
# zip1=zip(mask_3D,lgpt5_3D,Tmean,Tmax)
# task_list=[dask.delayed(LGPCalc.Eta_class)(m,l5,Tbar,Tmx,Txsnm) for m,l5,Tbar,Tmx in zip1]
# results=dask.compute(*task_list)
# eta_class=np.concatenate(results,axis=1)
lgpt5_3D=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis].astype('float16'),(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=bigchunk3D).to_delayed().ravel()
mask_3D=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=bigchunk3D).to_delayed().ravel()
Tmean=clim_reg.meanT_daily.rechunk(chunks=bigchunk3D).astype('float16').to_delayed().ravel()
Tmax=clim_reg.maxT_daily.rechunk(chunks=bigchunk3D).astype('float16').to_delayed().ravel()
zip1=zip(mask_3D,lgpt5_3D,Tmean,Tmax)
task_list=[dask.delayed(LGPCalc.Eta_class)(m,l5,Tbar,Tmx,Txsnm) for m,l5,Tbar,Tmx in zip1]
results=dask.compute(*task_list)
eta_class=np.concatenate(results,axis=1)

del lgpt5_3D,mask_3D,Tmean,Tmax,zip1,task_list,results

print('build task graph for islgp')
# islgp=da.where(clim_reg.meanT_daily>=5,np.int8(1),np.int8(0))
islgp=da.where(clim_reg.meanT_daily>=5,np.int8(1),np.int8(0)).rechunk(chunks=bigchunk3D)  #KLG
Fsnm = 5.5
kc_list = np.array([0.0, 0.1, 0.2, 0.5, 1.0],dtype='float32')

In [ ]:
# lgpt5=da.from_array(clim_reg.lgpt5,chunks=clim_reg.chunk2D)
# lgpt5_cl = client.scatter(lgpt5)
# # lgpt5=client.scatter(lgpt5)
# mask_cl=client.scatter(clim_reg.im_mask)
# istart0_cl = client.scatter(istart0)
# istart1_cl = client.scatter(istart1)  
# Sb_old_cl = client.scatter(da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32'))
# Wb_old_cl = client.scatter(da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32'))

# # # 3D chunked inputs
# Pet365_cl = client.scatter(da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D))
# p_cl = client.scatter(p)
# eta_class_cl=client.scatter(da.from_array(eta_class,chunks=clim_reg.chunk3D))
# Tx365_cl = client.scatter(clim_reg.maxT_daily)
# Pcp365_cl = client.scatter(clim_reg.totalPrec_daily)
# islgp_cl = client.scatter(islgp)


# lgpt5_cl=lgpt5
# mask_cl=clim_reg.im_mask
# istart0_cl = istart0
# istart1_cl = istart1  
# Sb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32')
# Wb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32')
# Pet365_cl = da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D)
# p_cl = psh(ng,pet)
# eta_class_cl=da.from_array(eta_class,chunks=clim_reg.chunk3D)
# Tx365_cl = clim_reg.maxT_daily
# Pcp365_cl = clim_reg.totalPrec_daily
# islgp_cl = islgp

lgpt5_cl=lgpt5
mask_cl=clim_reg.im_mask.rechunk(chunks=bigchunk2D)
istart0_cl = istart0
istart1_cl = istart1  
Sb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=bigchunk2D,dtype='float32')
Wb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=bigchunk2D,dtype='float32')
Pet365_cl = da.from_array(clim_reg.pet_daily,chunks=bigchunk3D)
p_cl = psh(ng,pet)
eta_class_cl=da.from_array(eta_class,chunks=bigchunk3D)
Tx365_cl = clim_reg.maxT_daily.rechunk(chunks=bigchunk3D)
Pcp365_cl = clim_reg.totalPrec_daily.rechunk(chunks=bigchunk3D)
islgp_cl = islgp

In [ ]:
# cluster=LocalCluster()#worker_dashboard_address='localhost:8787')#(log_directory=daskpath)
# # cluster=LocalCluster(n_workers=1, threads_per_worker=20)#, memory_limit='10GB')
# client=Client(cluster)
# client

In [ ]:
results=[]
task_list=[]
# for i in range(clim_reg.nchunks):
for i in range(nchunks):
    print('loop',i)
    mask_s=mask_cl.blocks[0,i].compute()
    Tx365_s=Tx365_cl.blocks[0,i,0].compute()
    islgp_s=islgp_cl.blocks[0,i,0].compute()
    Pcp365_s=Pcp365_cl.blocks[0,i,0].compute()
    Pet365_s=Pet365_cl.blocks[0,i,0].compute()
    Wb_old_s=Wb_old_cl.blocks[0,i].compute()
    Sb_old_s=Sb_old_cl.blocks[0,i].compute()
    istart0_s=istart0_cl.blocks[0,i].compute()
    istart1_s=istart1_cl.blocks[0,i].compute()
    lgpt5_s=lgpt5_cl.blocks[0,i].compute()
    # print('eta_class')
    eta_class_s=eta_class_cl.blocks[0,i].compute()
    # print('p')
    p_s=p_cl.blocks[0,i,0].compute()
    # print('lgp_tot')
    results.append(LGPCalc.EtaCalc(mask_s,Tx365_s,islgp_s,Pcp365_s,Txsnm,Fsnm,Pet365_s,Wb_old_s,Sb_old_s,istart0_s,istart1_s,Sa,D,p_s,kc_list,lgpt5_s,eta_class_s,clim_reg.doy_start,clim_reg.doy_end))

    # mask_s=dask.delayed(mask_cl.blocks[0,i])
    # Tx365_s=dask.delayed(Tx365_cl.blocks[0,i,0])
    # islgp_s=dask.delayed(islgp_cl.blocks[0,i,0])
    # Pcp365_s=dask.delayed(Pcp365_cl.blocks[0,i,0])
    # Pet365_s=dask.delayed(Pet365_cl.blocks[0,i,0])
    # Wb_old_s=dask.delayed(Wb_old_cl.blocks[0,i])
    # Sb_old_s=dask.delayed(Sb_old_cl.blocks[0,i])
    # istart0_s=dask.delayed(istart0_cl.blocks[0,i])
    # istart1_s=dask.delayed(istart1_cl.blocks[0,i])
    # lgpt5_s=dask.delayed(lgpt5_cl.blocks[0,i])
    # eta_class_s=dask.delayed(eta_class_cl.blocks[0,i])
    # p_s=dask.delayed(p_cl.blocks[0,i,0])
    # task_list.append(dask.delayed(LGPCalc.EtaCalc)(mask_s,Tx365_s,islgp_s,Pcp365_s,Txsnm,Fsnm,Pet365_s,Wb_old_s,Sb_old_s,istart0_s,istart1_s,Sa,D,p_s,kc_list,lgpt5_s,eta_class_s,clim_reg.doy_start,clim_reg.doy_end))
print('--------------------------------')
len(results),results[0].shape
# len(task_list),task_list[0]

In [ ]:
# results=dask.compute(*task_list)
# len(results),results[0].shape

In [ ]:
lgp_tot=np.concatenate(results,axis=1)
plt.imshow(lgp_tot,interpolation='none')
# plt.imshow(results[0],interpolation='none')
plt.colorbar()


In [ ]:
varlist=[mask_cl,Tx365_cl,islgp_cl,Pcp365_cl,Pet365_cl,Wb_old_cl,Sb_old_cl,istart0_cl,istart1_cl,lgpt5_cl,eta_class_cl,p_cl]
for vararr in varlist:
    if len(vararr.shape)==2:
        # vb=vararr.blocks[0,1].compute()
        # plt.imshow(vb,interpolation='none')
        plt.imshow(vararr,interpolation='none')
        plt.colorbar()
        plt.show()
    if len(vararr.shape)==3:
        # vb=vararr.blocks[0,1,0].compute()
        # plt.imshow(vb[:,:,0],interpolation='none')
        plt.imshow(vararr[:,:,1],interpolation='none')
        plt.colorbar()
        plt.show()       
        

In [ ]:
# mask_cl.blocks[0,1]
# Tx365_cl.blocks[0,1,0]
# islgp_cl.blocks[0,1,0]
# Pcp365_cl.blocks[0,1,0]
# Txsnm,Fsnm 
# Pet365_cl.blocks[0,1,0]
# Wb_old_cl.blocks[0,1]
Sb_old_cl.blocks[0,1]
istart0_cl.blocks[0,1]
istart1_cl.blocks[0,1]
Sa,D
p_cl.blocks[0,1,0]
kc_list
lgpt5_cl.blocks[0,1]
eta_class_cl.blocks[0,1,0]
clim_reg.doy_start,clim_reg.doy_end

# TRY SCATTERING ARRAYS
# TRY USING FUTURES ON p,istart0,istart1,eta_class

In [ ]:
# mask_s=client.scatter(mask_cl) 
# Tx365_s=client.scatter(Tx365_cl)
# islgp_s=client.scatter(islgp_cl)
# Pcp365_s=client.scatter(Pcp365_cl)
# Pet365_s=client.scatter(Pet365_cl)
# Wb_old_s=client.scatter(Wb_old_cl)
# Sb_old_s=client.scatter(Sb_old_cl)
# istart0_s=client.scatter(istart0_cl)
# istart1_s=client.scatter(istart1_cl)
# p_s=client.scatter(p_cl)
# lgpt5_s=client.scatter(lgpt5_cl)
# eta_class_s=client.scatter(eta_class_cl)

# mask_s=mask_cl) 
# Tx365_s=Tx365_cl.blocks[0,1,0]
# islgp_s=client.scatter(islgp_cl)
# Pcp365_s=client.scatter(Pcp365_cl)
# Pet365_s=client.scatter(Pet365_cl)
# Wb_old_s=client.scatter(Wb_old_cl)
# Sb_old_s=client.scatter(Sb_old_cl)
# istart0_s=client.scatter(istart0_cl)
# istart1_s=client.scatter(istart1_cl)
# p_s=client.scatter(p_cl)
# lgpt5_s=client.scatter(lgpt5_cl)
# eta_class_s=client.scatter(eta_class_cl)


# test=client.submit(LGPCalc.EtaCalc,mask_s, Tx365_s, islgp_s, Pcp365_s,Txsnm,Fsnm, Pet365_s, Wb_old_s, Sb_old_s, istart0_s, istart1_s,Sa,D, p_s, kc_list,lgpt5_s, eta_class_cl.blocks[0,1,0],clim_reg.doy_start,clim_reg.doy_end)
# test=client.submit(LGPCalc.EtaCalc,mask_cl.blocks[0,1], Tx365_cl.blocks[0,1,0], islgp_cl.blocks[0,1,0], Pcp365_cl.blocks[0,1,0],Txsnm,Fsnm, Pet365_cl.blocks[0,1,0], Wb_old_cl.blocks[0,1], Sb_old_cl.blocks[0,1], istart0_cl.blocks[0,1], istart1_cl.blocks[0,1],Sa,D, p_cl.blocks[0,1,0], kc_list,lgpt5_cl.blocks[0,1], eta_class_cl.blocks[0,1,0],clim_reg.doy_start,clim_reg.doy_end)
# test

In [ ]:
# # p_s=p_cl.blocks[0,1,0].compute()
# # p_cl = psh(da.zeros(clim_reg.pet_daily.shape,chunks=clim_reg.chunk3D,dtype='float32'), da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D))
# ng=da.zeros(clim_reg.pet_daily.shape,chunks=clim_reg.chunk3D,dtype='float32')
# pet=da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D)
# p_cl = psh(ng,pet)
# p_s=p_cl.blocks[0,1,0].compute()
# p_s#.dask

In [ ]:
p_s.nbytes/1E6

In [ ]:
def compute_var_chunk(var):
    return var.compute()

In [ ]:
# ng=da.zeros(clim_reg.pet_daily.shape,chunks=clim_reg.chunk3D,dtype='float32')
# pet=da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D)
# p_cl = psh(ng,pet)

In [ ]:
futures=[]
for i in range(2):
    print('loop',i)
    mask_s=client.submit(compute_var_chunk,mask_cl.blocks[0,i])
    Tx365_s=client.submit(compute_var_chunk,Tx365_cl.blocks[0,i,0])
    islgp_s=client.submit(compute_var_chunk,islgp_cl.blocks[0,i,0])
    Pcp365_s=client.submit(compute_var_chunk,Pcp365_cl.blocks[0,i,0])
    Pet365_s=client.submit(compute_var_chunk,Pet365_cl.blocks[0,i,0])
    Wb_old_s=client.submit(compute_var_chunk,Wb_old_cl.blocks[0,i])
    Sb_old_s=client.submit(compute_var_chunk,Sb_old_cl.blocks[0,i])
    istart0_s=client.submit(compute_var_chunk,istart0_cl.blocks[0,i])
    istart1_s=client.submit(compute_var_chunk,istart1_cl.blocks[0,i])
    lgpt5_s=client.submit(compute_var_chunk,lgpt5_cl.blocks[0,i])
    print('eta_class')
    eta_class_s=client.submit(compute_var_chunk,eta_class_cl.blocks[0,i])
    print('p')
    p_s=client.submit(compute_var_chunk,p_cl.blocks[0,i,0])

    print('lgp_tot')
    lgp_tot=client.submit(LGPCalc.EtaCalc,mask_s,Tx365_s,islgp_s,Pcp365_s,Txsnm,Fsnm,Pet365_s,Wb_old_s,Sb_old_s,istart0_s,istart1_s,Sa,D,p_s,kc_list,lgpt5_s,eta_class_s,clim_reg.doy_start,clim_reg.doy_end).result()
    futures.append(lgp_tot)
    print(futures)

futures


In [ ]:
futures

In [ ]:
lgpt5_s=lgpt5_cl.blocks[0,1].compute() 


In [ ]:
eta_class_s=eta_class_cl.blocks[0,1,0].compute()

In [ ]:
test=LGPCalc.EtaCalc(mask_s, Tx365_s, islgp_s, Pcp365_s,Txsnm,Fsnm, Pet365_s, Wb_old_s, Sb_old_s, istart0_s, istart1_s,Sa,D, p_s, kc_list,lgpt5_s, eta_class_s,clim_reg.doy_start,clim_reg.doy_end)
test

In [ ]:
futures=[]
for i in range(132):
    futures.append()
futuresclient.submit(LGPCalc.EtaCalc,mask_cl.blocks[0,i], Tx365_cl.blocks[0,i,0], islgp_cl.blocks[0,i,0], Pcp365_cl.blocks[0,i,0],Txsnm,Fsnm, Pet365_cl.blocks[0,i,0], Wb_old_cl.blocks[0,i], Sb_old_cl[0,i], istart0_cl[0,i], istart1_cl[0,i],Sa,D, p_cl[0,i,0], kc_list,lgpt5_cl[0,i], eta_class_cl[0,i,0],clim_reg.doy_start,clim_reg.doy_end)

# task_list=[client.submit(LGPCalc.EtaCalc,mask,Tx365,blgp,Pcp365,Txsnm,Fsnm,pet365,Wb_old,Sb_old,s0,s1,Sa,D,p_c,kc_list,lgp,eclass,clim_reg.doy_start,clim_reg.doy_end) for mask,Tx365,blgp,Pcp365,pet365,Wb_old,Sb_old,s0,s1,p_c,lgp,eclass in zipvars]

# futures=client.submit(LGPCalc.EtaCalc,mask_cl, Tx365_cl, islgp_cl, Pcp365_cl,Txsnm,Fsnm, Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl,Sa,D, p_cl, kc_list,lgpt5_cl, eta_class_cl,clim_reg.doy_start,clim_reg.doy_end)
# # mask_cl, Tx365_cl, islgp_cl, Pcp365_cl,Txsnm,Fsnm, Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl,Sa,D, p_cl, kc_list,lgpt5_cl, eta_class_cl,clim_reg.doy_start,clim_reg.doy_end
# # mask,Tx365,blgp,Pcp365,Txsnm,Fsnm,pet365,Wb_old,Sb_old,s0,s1,Sa,D,p_c,kc_list,lgp,eclass,clim_reg.doy_start,clim_reg.doy_end
# futures

In [ ]:
results=[]

In [ ]:
res=client.gather(futures)
res

In [ ]:
start=timer()
Sa=100.
D=1.
Txsnm = 0. 

print('build task graph for istart0,istart1,p')
# lgpt5=da.from_array(clim_reg.lgpt5,chunks=clim_reg.chunk2D)
istart0,istart1=LGPCalc.rainPeak(clim_reg.meanT_daily,lgpt5)
# pet_daily=clim_reg.pet_daily
p = LGPCalc.psh(da.zeros(clim_reg.pet_daily.shape,chunks=clim_reg.chunk3D,dtype='float32'), clim_reg.pet_daily)
# ndays=clim_reg.maxT_daily.shape[2]
# print(istart0,istart1,p)

# compute eta_class then chunk (reduces memory required in day loop)
print('compute eta_class')
# dask delayed using the default dask scheduler (no distributed) is fastest
lgpt5_3D=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis].astype('float16'),(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
mask_3D=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
Tmean=clim_reg.meanT_daily.astype('float16').to_delayed().ravel()
Tmax=clim_reg.maxT_daily.astype('float16').to_delayed().ravel()
zip1=zip(mask_3D,lgpt5_3D,Tmean,Tmax)
task_list=[dask.delayed(LGPCalc.Eta_class)(m,l5,Tbar,Tmx,Txsnm) for m,l5,Tbar,Tmx in zip1]
results=dask.compute(*task_list)
eta_class=np.concatenate(results,axis=1)
del lgpt5_3D,mask_3D,Tmean,Tmax,zip1,task_list,results



eta_class_cl=da.from_array(eta_class,chunks=clim_reg.chunk3D)#.to_delayed().ravel()
eta_class_cl



# compute eta_class with default dask arrays is too slow
# dask futures scatter/submit/gather returns weird shape

print('build task graph for islgp')
islgp=da.where(clim_reg.meanT_daily>=5,np.int8(1),np.int8(0))  #KLG
Fsnm = 5.5
kc_list = np.array([0.0, 0.1, 0.2, 0.5, 1.0],dtype='float32')

# # smallchunk3D=(-1,4,-1)#self.chunk3D#(-1,400,-1)
# # smallchunk2D=(-1,4)#self.chunk2D#(-1,400)

# # # 2D chunked inputs
# # mask_cl=clim_reg.im_mask.rechunk(chunks=smallchunk2D).to_delayed().ravel()#.blocks.ravel()
# # lgpt5_cl = lgpt5.rechunk(chunks=smallchunk2D).to_delayed().ravel()#.blocks.ravel()
# # istart0_cl = istart0.rechunk(chunks=smallchunk2D).to_delayed().ravel()#.blocks.ravel()
# # istart1_cl = istart1.rechunk(chunks=smallchunk2D).to_delayed().ravel()#.blocks.ravel()       
# # Sb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=smallchunk2D,dtype='float32').to_delayed().ravel()#.blocks.ravel()
# # Wb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=smallchunk2D,dtype='float32').to_delayed().ravel()#.blocks.ravel() 
# print('prepare delayed input chunks')
# lgpt5=da.from_array(clim_reg.lgpt5,chunks=clim_reg.chunk2D)
# mask_cl=clim_reg.im_mask.to_delayed().ravel()
# lgpt5_cl = lgpt5.to_delayed().ravel()
# istart0_cl = istart0.to_delayed().ravel()
# istart1_cl = istart1.to_delayed().ravel()    
# Sb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32').to_delayed().ravel()
# Wb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32').to_delayed().ravel()

# # # 3D chunked inputs
# # Pet365_cl = pet_daily.rechunk(chunks=smallchunk3D).to_delayed().ravel()#.blocks.ravel()
# # p_cl = p.rechunk(chunks=smallchunk3D).to_delayed().ravel()#.blocks.ravel()
# # eta_class_cl = eta_class.rechunk(chunks=smallchunk3D).to_delayed().ravel()#.blocks.ravel()
# # Tx365_cl = clim_reg.maxT_daily.rechunk(chunks=smallchunk3D).to_delayed().ravel()#.blocks.ravel()
# # Pcp365_cl = clim_reg.totalPrec_daily.rechunk(chunks=smallchunk3D).to_delayed().ravel()#.blocks.ravel()
# # islgp_cl = islgp.rechunk(chunks=smallchunk3D).to_delayed().ravel()
# Pet365_cl = da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D).to_delayed().ravel()
# p_cl = p.to_delayed().ravel()
# eta_class_cl=da.from_array(eta_class,chunks=clim_reg.chunk3D).to_delayed().ravel()
# Tx365_cl = clim_reg.maxT_daily.to_delayed().ravel()
# Pcp365_cl = clim_reg.totalPrec_daily.to_delayed().ravel()
# islgp_cl = islgp.to_delayed().ravel()


# # # zip all the chunked variables together
# zipvars=zip(mask_cl, Tx365_cl, islgp_cl, Pcp365_cl, Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl, p_cl, lgpt5_cl, eta_class_cl)
# task_time=timer()-start
# print('time spent on prepping vars',task_time) 

# print('computing LGP')
# start=timer()
# task_list=[dask.delayed(LGPCalc.EtaCalc,pure=True)(mask,Tx365,blgp,Pcp365,Txsnm,Fsnm,pet365,Wb_old,Sb_old,s0,s1,Sa,D,p_c,kc_list,lgp,eclass,clim_reg.doy_start,clim_reg.doy_end) for mask,Tx365,blgp,Pcp365,pet365,Wb_old,Sb_old,s0,s1,p_c,lgp,eclass in zipvars]
# results=dask.compute(*task_list)
# task_time=timer()-start
# print('NUMBER OF TASKS',len(task_list))
# print('time spent computing lgp_tot chunks',task_time) 
# print('free RAM',psutil.virtual_memory().free/1E9)

# print('aggregating lgp_tot')
# start=timer()
# lgp_tot=np.concatenate(results,axis=1)
# task_time=timer()-start
# print('time spent on lgp_tot concat',task_time)


In [ ]:
# from dask.distributed import Client#,LocalCluster
cluster=LocalCluster(n_workers=1, threads_per_worker=10, memory_limit='10GB')
client=Client(cluster)
# client=Client()
client

In [ ]:
start=timer()
Sa=100.
D=1.

lgpt5=da.from_array(clim_reg.lgpt5,chunks=clim_reg.chunk2D)#.persist()
istart0,istart1=LGPCalc.rainPeak(clim_reg.meanT_daily,lgpt5)
# istart0=istart0.persist()
# istart1=istart1.persist()
pet_daily=clim_reg.pet_daily#.persist()
p = LGPCalc.psh(da.zeros(pet_daily.shape,chunks=clim_reg.chunk3D,dtype='float32'), pet_daily)#.persist()
ndays=clim_reg.maxT_daily.shape[2]


lgpt5_3D=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis].astype('float16'),(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
mask_3D=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
Tmean=clim_reg.meanT_daily.to_delayed().ravel()
Tmax=clim_reg.maxT_daily.to_delayed().ravel()
zip1=zip(mask_3D,lgpt5_3D,Tmean,Tmax)
Txsnm = 0. 
task_list=[dask.delayed(LGPCalc.Eta_class)(m,l5,Tbar,Tmx,Txsnm) for m,l5,Tbar,Tmx in zip1]
results=dask.compute(*task_list)
eta_class=np.concatenate(results,axis=1)
del lgpt5_3D,mask_3D,Tmean,Tmax,zip1,task_list,results
eta_class_cl=da.from_array(eta_class,chunks=clim_reg.chunk3D).to_delayed().ravel()



# res_list=dask.compute(*eta_class_cl)
# eta_class_list=[da.from_delayed(class_c,(clim_reg.im_height,clim_reg.im_width),dtype='int8') for class_c in eta_class_cl]
# eta_class=np.concatenate(res_list,axis=1)
# eta_class=dask.delayed(eta_class)

# zip1=zip(Tmean_cl,Tmax_cl)
# res=[dask.delayed(LGPCalc.Eta_class, pure=True)(mask_delay,lgpt5_delay,Tbar,Tmax,Txsnm) for Tbar,Tmax in zip1]
# eta_class_list=[da.from_delayed(res_c,(clim_reg.im_height,clim_reg.im_width),dtype='int32') for res_c in res]
# eta_class=da.stack(eta_class_list,axis=-1)#.rechunk(chunks=(-1,12,365))
# eta_class=LGPCalc.Eta_class(mask_3D,lgpt5_3D,clim_reg.meanT_daily,clim_reg.maxT_daily,Txsnm)
islgp=da.where(clim_reg.meanT_daily>=5,np.int8(1),np.int8(0))#.persist()  #KLG
Fsnm = 5.5
kc_list = np.array([0.0, 0.1, 0.2, 0.5, 1.0],dtype='float32')

# smallchunk3D=(-1,4,-1)#self.chunk3D#(-1,400,-1)
# smallchunk2D=(-1,4)#self.chunk2D#(-1,400)
#constants
Txsnm_cl=da.from_array(np.zeros(clim_reg.nchunks),chunks=(1))
Fsnm_cl=da.from_array(np.zeros(clim_reg.nchunks)+Fsnm,chunks=(1))
Sa_cl=da.from_array(np.zeros(clim_reg.nchunks)+Sa,chunks=(1))
D_cl=da.from_array(np.ones(clim_reg.nchunks),chunks=(1))
kclist_cl=da.broadcast_to(kc_list[:,np.newaxis],(kc_list.shape[0],clim_reg.nchunks)).rechunk(chunks=(kc_list.shape[0],1))
dstart_cl=da.from_array(np.zeros(clim_reg.nchunks)+clim_reg.doy_start,chunks=(1))
dend_cl=da.from_array(np.zeros(clim_reg.nchunks)+clim_reg.doy_end,chunks=(1))


# 2D chunked inputs
mask_cl=clim_reg.im_mask#.to_delayed().ravel()#.blocks.ravel()
lgpt5_cl = lgpt5#.to_delayed().ravel()#.blocks.ravel()
istart0_cl = istart0#.to_delayed().ravel()#.blocks.ravel()
istart1_cl = istart1#.to_delayed().ravel()#.blocks.ravel()       
Sb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32')#.to_delayed().ravel()#.blocks.ravel()
Wb_old_cl = da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32')#.to_delayed().ravel()#.blocks.ravel() 

# 3D chunked inputs
Pet365_cl = pet_daily#.to_delayed().ravel()#.blocks.ravel()
p_cl = p#.to_delayed().ravel()#.blocks.ravel()
# eta_class_cl = eta_class.to_delayed().ravel()#.blocks.ravel()
eta_class_cl=eta_class#.to_delayed().ravel()
Tx365_cl = clim_reg.maxT_daily#.to_delayed().ravel()#.blocks.ravel()
Pcp365_cl = clim_reg.totalPrec_daily#.to_delayed().ravel()#.blocks.ravel()
islgp_cl = islgp#.to_delayed().ravel()

# # 2D chunked inputs
# mask_cl=client.scatter(clim_reg.im_mask)#.to_delayed().ravel())#.blocks.ravel()
# lgpt5_cl = client.scatter(lgpt5)#.to_delayed().ravel())#.blocks.ravel()
# istart0_cl = client.scatter(istart0)#.to_delayed().ravel())#.blocks.ravel()
# istart1_cl = client.scatter(istart1)#.to_delayed().ravel())#.blocks.ravel()       
# Sb_old_cl = client.scatter(da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32'))#.to_delayed().ravel())#.blocks.ravel()
# Wb_old_cl = client.scatter(da.zeros((clim_reg.im_height,clim_reg.im_width),chunks=clim_reg.chunk2D,dtype='float32'))#.to_delayed().ravel())#.blocks.ravel() 

# # 3D chunked inputs
# Pet365_cl = client.scatter(pet_daily)#.to_delayed().ravel())#.blocks.ravel()
# p_cl = client.scatter(p)#.to_delayed().ravel())#.blocks.ravel()
# # eta_class_cl = eta_class.to_delayed().ravel()#.blocks.ravel()
# eta_class_cl=client.scatter(eta_class)#.to_delayed().ravel())
# Tx365_cl = client.scatter(clim_reg.maxT_daily)#.to_delayed().ravel())#.blocks.ravel()
# Pcp365_cl = client.scatter(clim_reg.totalPrec_daily)#.to_delayed().ravel())#.blocks.ravel()
# islgp_cl = client.scatter(islgp)#.to_delayed().ravel())

# zip all the chunked variables together
# zipvars=zip(mask_cl, Tx365_cl, islgp_cl, Pcp365_cl, Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl, p_cl, lgpt5_cl, eta_class_cl)
# zipvars=zip(mask_cl, Tx365_cl, islgp_cl, Pcp365_cl, Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl, p_cl, lgpt5_cl, eta_class_cl)
# print(len(mask_cl.chunks[1]),len(Tx365_cl.chunks[1]),len(islgp_cl.chunks[1]),len(Pcp365_cl.chunks[1]),len(Txsnm_cl.chunks[0]),len(Fsnm_cl.chunks[0]),len(Pet365_cl.chunks[1]),len(Wb_old_cl.chunks[1]),len(Sb_old_cl.chunks[1]))
# print(len(istart0_cl.chunks[1]),len(istart1_cl.chunks[1]),len(Sa_cl.chunks[0]),len(D_cl.chunks[0]),len(p_cl.chunks[1]),len(kclist_cl.chunks[1]),len(lgpt5_cl.chunks[1]),len(eta_class_cl.chunks[1]),len(dstart_cl.chunks[0]),len(dend_cl.chunks[0])  )#, Tx365_cl, islgp_cl, Pcp365_cl,Txsnm_cl,Fsnm_cl,Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl,Sa_cl,D_cl,p_cl,kclist_cl,lgpt5_cl, eta_class_cl,dstart_cl,dend_cl)
print(mask_cl.numblocks, Tx365_cl.numblocks, islgp_cl.numblocks, Pcp365_cl.numblocks,Txsnm_cl.numblocks,Fsnm_cl.numblocks,Pet365_cl.numblocks, Wb_old_cl.numblocks, Sb_old_cl.numblocks)
print(istart0_cl.numblocks, istart1_cl.numblocks,Sa_cl.numblocks,D_cl.numblocks,p_cl.numblocks,kclist_cl.numblocks,lgpt5_cl.numblocks, eta_class_cl.numblocks,dstart_cl.numblocks,dend_cl.numblocks)
zipvars=zip(mask_cl, Tx365_cl, islgp_cl, Pcp365_cl,Txsnm_cl,Fsnm_cl,Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl,Sa_cl,D_cl,p_cl,kclist_cl,lgpt5_cl, eta_class_cl,dstart_cl,dend_cl)
task_time=timer()-start
print('time spent on prepping vars',task_time) 


In [ ]:
print(da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).chunks)
print(da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).chunksize)
print(clim_reg.meanT_daily.chunksize)
print(clim_reg.maxT_daily.chunksize)

In [ ]:
# lgpt5_3D=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D)
# mask_3D=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D)
# Tmean=clim_reg.meanT_daily
# Tmax=clim_reg.maxT_daily
# Tmax

def ETA_CLASS(mask,lg5,ta,tx,tmelt):
    eta_class=np.zeros(tx.shape,dtype='int8')
    # category 1, snow
    inds = (ta<=0)
    inds &= (tx<=tmelt)
    eta_class[inds]=1
    # category 2, snow melting
    inds = (ta<=0)
    inds &= (tx>=0)
    inds &= (eta_class==0)
    eta_class[inds]=2
    # category 3, cold pre growing
    inds = (ta>0)
    inds &= (ta<5)
    inds &= (eta_class==0)
    eta_class[inds]=3
    # category 4, warm
    inds = (lg5<365)
    inds &= (ta>=5)
    inds &= (eta_class==0)
    eta_class[inds]=4
    # category 5, warmest
    inds = (mask==1)
    inds &= (eta_class==0)
    eta_class[inds]=5
    return eta_class


In [ ]:
def test(var1,var2):
    return var1,var2

In [ ]:
print('free RAM',psutil.virtual_memory().free/1E9)
start=timer()
Txsnm = 0. 
lgpt5_3D=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis].astype('float16'),(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
mask_3D=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
Tmean=clim_reg.meanT_daily.to_delayed().ravel()
Tmax=clim_reg.maxT_daily.to_delayed().ravel()
zip1=zip(mask_3D,lgpt5_3D,Tmean,Tmax)
task_list=[dask.delayed(LGPCalc.Eta_class)(m,l5,Tbar,Tmx,Txsnm) for m,l5,Tbar,Tmx in zip1]
print(len(task_list))
results=dask.compute(*task_list)
task_time=timer()-start
print('time:',task_time)
print('free RAM',psutil.virtual_memory().free/1E9)
eta_class=np.concatenate(results,axis=1)
print(eta_class.nbytes/1E9)
eta_class_cl=da.from_array(eta_class,chunks=clim_reg.chunk3D)#.to_delayed().ravel()
del lgpt5_3D,mask_3D,Tmean,Tmax,zip1,task_list,results
eta_class_cl


# print('free RAM',psutil.virtual_memory().free/1E9)
# start=timer()
# Txsnm = 0.
# tmelt=0. 
# lg5=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis].astype('float16'),(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D)
# mask=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D)
# ta=clim_reg.meanT_daily
# tx=clim_reg.maxT_daily
# # eta_class=LGPCalc.Eta_class(mask,lg5,ta,tx,Txsnm).compute()
# # eta_class=da.ones(ta.shape,dtype='int8',chunks=clim_reg.chunk3D)
# # category 1, snow
# inds = (ta<=0)
# # inds &= (tx<=tmelt)
# # eta_class[inds]=1
# # # category 2, snow melting
# # inds = (ta<=0)
# # inds &= (tx>=0)
# # inds &= (eta_class==0)
# # eta_class[inds]=2
# # # category 3, cold pre growing
# # inds = (ta>0)
# # inds &= (ta<5)
# # inds &= (eta_class==0)
# # eta_class[inds]=3
# # # category 4, warm
# # inds = (lg5<365)
# # inds &= (ta>=5)
# # inds &= (eta_class==0)
# # eta_class[inds]=4
# # # category 5, warmest
# # inds = (mask==1)
# # inds &= (eta_class==0)
# # eta_class[inds]=5

# inds=inds.compute()
# # eta_class=eta_class.compute()
# task_time=timer()-start
# print('time:',task_time)
# print('free RAM',psutil.virtual_memory().free/1E9)
# # # # eta_class=np.concatenate(results,axis=1)
# # # del lgpt5_3D,mask_3D,Tmean,Tmax
# # print(eta_class.nbytes/1E9)
# # eta_class
# inds
# # print(lgpt5_3D.dtype,lgpt5_3D.nbytes/1E9)
# # lgpt5_3D
# # print(mask_3D.dtype,mask_3D.nbytes/1E9)
# # mask_3D


# lgpt5_3D=da.broadcast_to(clim_reg.lgpt5[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
# mask_3D=da.broadcast_to(clim_reg.im_mask[:,:,np.newaxis],(clim_reg.im_height,clim_reg.im_width,365)).rechunk(chunks=clim_reg.chunk3D).to_delayed().ravel()
# Tmean=clim_reg.meanT_daily.to_delayed().ravel()
# Tmax=clim_reg.maxT_daily.to_delayed().ravel()
# zip1=zip(mask_3D,lgpt5_3D,Tmean,Tmax)
# # Txsnm = 0. 
# eta_class_res=[dask.delayed(LGPCalc.Eta_class)(m,l5,Tbar,Tmx,Txsnm) for m,l5,Tbar,Tmx in zip1]
# eta_class_list=[da.from_delayed(res_c,(clim_reg.chunk3D),dtype='int8') for res_c in eta_class_res]
# eta_class=da.concatenate(eta_class_list,axis=1)
# eta_class_cl=eta_class.to_delayed().ravel()
# eta_class_cl

In [ ]:
# # test=dask.compute(*eta_class_cl)
# test=eta_class.compute()
# test

In [ ]:
# etc=np.concatenate(test,axis=1)
# etc.shape,etc.dtype,etc.nbytes/1E9

In [ ]:
# def test_func(mask_cl, Tx365_cl, islgp_cl, Pcp365_cl, Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl, p_cl, lgpt5_cl, eta_class_cl):
#     return mask_cl, Tx365_cl, islgp_cl, Pcp365_cl, Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl, p_cl, lgpt5_cl, eta_class_cl

# def test_func(var_cl):
#     return var_cl

In [ ]:
start=timer()
task_list=[dask.delayed(LGPCalc.EtaCalc,pure=True)(mask,Tx365,blgp,Pcp365,Txsnm,Fsnm,pet365,Wb_old,Sb_old,s0,s1,Sa,D,p_c,kc_list,lgp,eclass,clim_reg.doy_start,clim_reg.doy_end) for mask,Tx365,blgp,Pcp365,pet365,Wb_old,Sb_old,s0,s1,p_c,lgp,eclass in zipvars]
# task_list=client.persist(task_list)
# task_list=[dask.delayed(LGPCalc.EtaCalc,pure=True)(mask,Tx365,blgp,Pcp365,Txsnm,Fsnm,pet365,Wb_old,Sb_old,s0,s1,Sa,D,p_c,kc_list,lgp,eta_class,clim_reg.doy_start,clim_reg.doy_end) for mask,Tx365,blgp,Pcp365,pet365,Wb_old,Sb_old,s0,s1,p_c,lgp in zipvars]
# task_list=[dask.delayed(test_func,pure=True)(mask,Tx365,blgp,Pcp365,pet365,Wb_old,Sb_old,s0,s1,p_c,lgp,eclass) for mask,Tx365,blgp,Pcp365,pet365,Wb_old,Sb_old,s0,s1,p_c,lgp,eclass in zipvars]
# task_list=[dask.delayed(test_func,pure=True)(var) for var in eta_class]
results=dask.persist(*task_list)
task_time=timer()-start
print('NUMBER OF TASKS',len(task_list))
print('time spent building task list',task_time) 
print('free RAM',psutil.virtual_memory().free/1E9)


In [ ]:
start=timer()
# results=dask.compute(*task_list, scheduler='single-threaded',num_workers=20)#, optimize_graph=False,scheduler='processes')
# results=dask.compute(*task_list)#,scheduler='distributed')#, scheduler='synchronous')#,num_workers=20)#, optimize_graph=False,scheduler='processes')
# results=dask.persist(*task_list)#,scheduler='distributed')#, scheduler='synchronous')#,num_workers=20)#, optimize_graph=False,scheduler='processes')
   
# lgp_tot=client.submit(LGPCalc.EtaCalc,mask_cl, Tx365_cl, islgp_cl, Pcp365_cl,Txsnm,Fsnm,Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl,Sa,D,p_cl,kc_list,lgpt5_cl, eta_class_cl,clim_reg.doy_start,clim_reg.doy_end).result()
# futures=client.map(LGPCalc.EtaCalc,*zipvars).result()
lgp_tot=client.map(LGPCalc.EtaCalc,*[mask_cl, Tx365_cl, islgp_cl, Pcp365_cl,Txsnm_cl,Fsnm_cl,Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl,Sa_cl,D_cl,p_cl,kclist_cl,lgpt5_cl, eta_class_cl,dstart_cl,dend_cl])
# lgp_tot=[f.result() for f in futures]

# automated dask is too slow on dask arrays
# lgp_tot=LGPCalc.EtaCalc(mask_cl, Tx365_cl, islgp_cl, Pcp365_cl,Txsnm,Fsnm,Pet365_cl, Wb_old_cl, Sb_old_cl, istart0_cl, istart1_cl,Sa,D,p_cl,kc_list,lgpt5_cl, eta_class_cl,clim_reg.doy_start,clim_reg.doy_end)
# our function and inputs are too complicated for map blocks
# lgp_tot=da.map_blocks(LGPCalc.EtaCalc,mask_cl,Tx365_cl,islgp_cl,Pcp365_cl,Txsnm,Fsnm,Pet365_cl,Wb_old_cl,Sb_old_cl,istart0_cl,istart1_cl,Sa,D,p_cl,kc_list,lgpt5_cl,eta_class_cl,clim_reg.doy_start,clim_reg.doy_end)
task_time=timer()-start
print('time spent in compute',task_time)    

In [ ]:
print(psutil.virtual_memory().free/1E9)
lgp_tot

In [ ]:
def concat_res(var):
    var_concat=np.concatenate(var,axis=1)
    return var_concat

# ans=client.gather(lgp_tot)
ans=client.submit(concat_res,lgp_tot)
ans

In [ ]:
total=client.gather(ans)
total

In [ ]:
results[0]

In [ ]:
start=timer()
lgp_tot=np.concatenate(results,axis=1)
task_time=timer()-start
print('time spent on lgp_tot concat',task_time)

In [ ]:
plt.imshow(lgp_tot,interpolation='none')
plt.colorbar()

In [ ]:
len(ETA), ETA[0].shape
# plt.imshow(ETM[:,:,0],interpolation='none')
# plt.colorbar()

In [ ]:
test=np.concatenate(ETA,axis=1)
test.shape

In [ ]:
plt.imshow(test[:,:,2],interpolation='none')
plt.colorbar()

In [ ]:
tlgpt5=da.from_array(clim_reg.lgpt5,chunks=clim_reg.chunk2D)
plt.imshow(tlgpt5,interpolation='none')
plt.colorbar()

In [ ]:
istart0,istart1=LGPCalc.rainPeak(clim_reg.meanT_daily,tlgpt5)
plt.imshow(istart0,interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(lgp,interpolation='none')
plt.colorbar()

In [ ]:
pet_daily=da.from_array(clim_reg.pet_daily,chunks=clim_reg.chunk3D)
p = LGPCalc.psh(da.zeros(pet_daily.shape,chunks=clim_reg.chunk3D,dtype='float32'), pet_daily)
plt.imshow(p[:,:,0],interpolation='none')
plt.colorbar()

In [ ]:
ndays=clim_reg.maxT_daily.shape[2]
lgpt5_3D=da.tile(clim_reg.lgpt5[:,:,np.newaxis],(1,1,ndays)).rechunk(chunks=clim_reg.chunk3D) 
mask_3D=da.tile(clim_reg.im_mask[:,:,np.newaxis],(1,1,ndays)).rechunk(chunks=clim_reg.chunk3D)
Txsnm = 0. 
eta_class=LGPCalc.Eta_class(mask_3D,lgpt5_3D,clim_reg.meanT_daily,clim_reg.maxT_daily,Txsnm)
plt.imshow(eta_class[:,:,100],interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(clim_reg.maxT_daily[:,:,0],interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(clim_reg.meanT_daily[:,:,0],interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(clim_reg.totalPrec_daily[:,:,0],interpolation='none')
plt.colorbar()

In [ ]:
# var=clim_reg.maxT_daily.compute()[:,0:400,:]
# var=np.append(var, var[:,:,0:30],axis=2)
# print(var.shape,var.dtype)

# start=timer()
# rollmean=LGPCalc.val10day(var)
# task_time=timer()-start
# print(task_time)


var=ETA[0]
print(var.shape,var.dtype)

start=timer()
rollmean=LGPCalc.val10day(var)
task_time=timer()-start
print(task_time)

In [ ]:
import scipy

In [ ]:
    # window_size = 10
    # # first reshape the input data to 2d (space, time)
    # Et2d=np.reshape(Et,(Et.shape[0]*Et.shape[1],Et.shape[2])).astype('float32')
    # # use apply_along_axis to apply a rolling mean with length=window_size
    # rollmean2d=np.apply_along_axis(lambda m: np.convolve(m, np.ones(window_size), mode='valid')/window_size, axis=1, arr=Et2d) 
    # # reshape the result back to 3 dimensions
    # rollmean3d=np.reshape(rollmean2d,(Et.shape[0],Et.shape[1],rollmean2d.shape[1])).astype('float32')
start=timer()
window_size=10
scipyroll=scipy.signal.convolve(ETA[0],np.ones((1,1,window_size)),mode='valid',method='fft')/window_size
task_time=timer()-start
print(task_time)
scipyroll.shape

In [ ]:
plt.imshow(rollmean[:,:,0],interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow(roll[:,:,0],interpolation='none')
plt.colorbar()

In [ ]:
plt.imshow((rollmean[:,:,0]-roll[:,:,0]),interpolation='none')
plt.colorbar()

In [ ]:
test=np.arange(20)

testfull=np.tile(test,(428,400,1))
testfull.shape

In [ ]:
def moving_average(a, n=10):
    ret = np.cumsum(a, axis=-1, dtype='float32')
    ret[:,:,n:] = ret[:,:,n:] - ret[:,:,:-n]
    return ret[:,:,n - 1:] / n

In [ ]:
roll=moving_average(ETA[0])
# roll

In [ ]:
roll.dtype

In [ ]:
lgpt5_delayed=da.from_array(clim_reg.lgpt5,chunks=clim_reg.chunk2D).blocks.ravel()
lgpt5_delayed

In [ ]:
plt.imshow(clim_reg.Etm365[:,:,1],interpolation='none')
plt.colorbar()

In [ ]:
lgp#[0].shape

In [ ]:
plt.imshow(lgp,interpolation='none')
plt.colorbar()

In [ ]:
start=timer()

tclimate = clim_reg.getThermalClimate()

task_time=timer()-start
task_time

In [ ]:
clim_reg.__dict__.keys()
# clim_reg.im_mask,clim_reg.elevation

In [ ]:
# plt.imshow(clim_reg.meanT_monthly_sealevel[:,:,2],interpolation='none')
# plt.colorbar()

In [ ]:
clim_reg.maxT_daily.transpose((2,0,1)).rechunk(chunks=(1,-1,-1)).blocks.ravel()

In [ ]:
import psutil
psutil.virtual_memory().free/1E9

In [ ]:
test=[]

In [ ]:
# start=timer()

# istart0,istart1 = clim_reg.getLGP(Sa=100., D=1.)

# task_time=timer()-start
# task_time

In [ ]:
def psh(ng, et0):
    # psh0=np.where(ng==0,np.float32(0.5),np.float32(0.3+(ng-1)*.05))
    # psh = psh0 + .04 * (5.-et0)
    # psh=np.where(psh<0.1,np.float32(0.1),np.float32(psh))  
    # psh=np.where(psh>0.8,np.float32(0.8),np.float32(psh))
    psh0=np.where(ng==0,0.5,0.3+(ng-1)*.05)
    psh = psh0 + .04 * (5.-et0)
    psh=np.where(psh<0.1,0.1,psh)  
    psh=np.where(psh>0.8,0.8,psh)  
    return psh

In [ ]:
def rainPeak(meanT_daily, lgpt5):
    day1=np.argmax(meanT_daily>=5.0,axis=2)   #KLG
    day1=np.where(lgpt5==0,np.nan,day1)  #KLG
    istart0=np.where((lgpt5<365),day1,0).astype('float32') # replaces if block  #KLG
    dat1=np.where(istart0>365,istart0-365,istart0)  # replaces setdat function  #KLG
    istart1=np.where((lgpt5<365),dat1+lgpt5-1,lgpt5-1).astype('float32') # replaces if block  #KLG
    return istart0, istart1

In [ ]:
# start=timer()

# p = psh(np.zeros(clim_reg.pet_daily[:,:,0].shape,dtype='float32'), clim_reg.pet_daily[:,:,0])   #KLG
# # p = psh(np.zeros(clim_reg.pet_daily.shape,dtype='float32'), clim_reg.pet_daily)   #KLG

# task_time=timer()-start
# task_time

# create delayed chunks
lgpt5_delayed=da.from_array(lgpt5,chunks=clim_reg.chunk2D).to_delayed().ravel()
meanT_delayed=clim_reg.meanT_daily.to_delayed().ravel()

# create list of delayed compute tasks
task_list=[dask.delayed(rainPeak)(meanT_chunk,lgp_chunk) for meanT_chunk,lgp_chunk in zip(meanT_delayed,lgpt5_delayed)]

# compute
results_list=dask.compute(*task_list)

# separate results into lists of chunks
istart0_chunks=[]
istart1_chunks=[]
for c in range(clim_reg.nchunks):
    istart0_chunks.append(results_list[c][0])
    istart1_chunks.append(results_list[c][1])

# # concatenate to numpy array
# istart0=np.concatenate(istart0_chunks,axis=1)
# istart1=np.concatenate(istart1_chunks,axis=1)

istart0_delayed=[]
for arr in istart0_chunks:
    istart0_delayed.append(dask.delayed(arr))

istart0_delayed

In [ ]:
test=dask.delayed(clim_reg.meanT_daily)
test

In [ ]:
test=clim_reg.meanT_daily.to_delayed().ravel()
test

In [ ]:
timechunks=(1,-1,clim_reg.chunk3D[1])
Tx365 = clim_reg.maxT_daily.transpose((2,0,1)).rechunk(chunks=timechunks).blocks.ravel()
len(Tx365),Tx365[0]

In [ ]:
len(Tx365[0:clim_reg.nchunks])
istart=clim_reg.nchunks*i
iend=istart+clim_reg.nchunks
# clim_reg.maxT_daily.transpose((2,0,1)).rechunk(chunks=timechunks).blocks.ravel()

In [ ]:
s=(3,5)

arr1=np.array([[1,2,3,4,5],
               [6,7,8,9,10],
               [11,12,13,14,15]],dtype='int8')

arr2=-np.array([[1,2,3,4,5],
               [6,7,8,9,10],
               [11,12,13,14,15]],dtype='int8')

# np0=np.zeros(s,dtype='int8')
# np1=np.ones(s,dtype='int8')

# nptest=np.stack((np0,np1),axis=-1,dtype='int8')
# nptest

# nptest=np.stack((arr1,arr2),axis=-1,dtype='int8')
# darr=da.from_array(nptest,chunks=(3,2,1))
# nptest.shape,nptest
nptest=np.stack((arr1,arr2),axis=0,dtype='int8')
darr=da.from_array(nptest,chunks=(1,-1,2))
darr


In [ ]:
darr.blocks.shape

In [ ]:
darr.blocks[0].shape

In [ ]:
for i,block in enumerate(darr.blocks.ravel()[0:3]):
    print('block',i)
    print(block.compute())
    print('---')

In [ ]:
test=dask.compute(*darr.blocks.ravel()[0:3])
test2=np.concatenate(test,axis=2)
test2.shape

In [ ]:
test2

In [ ]:
nptest.shape,nptest

In [ ]:
np.stack((arr1,arr2),axis=2,dtype='int8')

In [ ]:
nptest.transpose((1,2,0))

In [ ]:
# parallel=True; for global expect ~1min (laptop), ~1.5min (desktop) run time 
# parallel=True; for china expect x (laptop), ~1s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time

start=timer()

tclimate = clim_reg.getThermalClimate()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(tclimate, cmap=plt.get_cmap('gist_ncar_r', 12),vmin=-0.3,vmax=12.5,interpolation='none')
plt.title('Thermal Climate')
plt.colorbar(shrink=0.4)
plt.savefig(out_path+"ThermalClimate.png",bbox_inches ="tight",dpi=300) #Save as PNG image
plt.show()

In [ ]:
# parallel=True; for global expect ~3s (laptop), ~4s (desktop) run time 
# parallel=True; for china expect x (laptop), <1s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time
start=timer()

tzone = clim_reg.getThermalZone()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(tzone, cmap=plt.get_cmap('gist_ncar_r', 12),vmin=-0.3,vmax=12.5,interpolation='none')
plt.title('Thermal Zones')
plt.colorbar(shrink=0.4)
plt.savefig(out_path+"ThermalZone"+revname+".png",bbox_inches ="tight",dpi=300) #Save as PNG image
plt.show()

In [ ]:
# parallel=True; for global expect ~1min (laptop), ~1.25min (desktop) run time 
# parallel=True; for china expect x (laptop), ~1.5s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time
start=timer()

lgpt0 = clim_reg.getThermalLGP0()
lgpt5 = clim_reg.getThermalLGP5()
lgpt10 = clim_reg.getThermalLGP10()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
#======================
# plt.figure(figsize=(20, 4),dpi=300)
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(lgpt0,vmin=0,vmax=366,interpolation='none')
plt.title('LGPt 0')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 2)
plt.imshow(lgpt5, vmin=0, vmax=366,interpolation='none')
plt.title('LGPt 5')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 3)
plt.imshow(lgpt10, vmin=0, vmax=366,interpolation='none')
plt.title('LGPt 10')
plt.colorbar(shrink=0.8)
#----------------------
plt.savefig(out_path+"ThermalLGPs"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
#======================

In [ ]:
# parallel=True; for global expect ~70s (laptop), ~75s (desktop) run time 
# parallel=True; for china expect x (laptop), ~1.5s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time
start=timer()

tsum0 = clim_reg.getTemperatureSum0()
tsum5 = clim_reg.getTemperatureSum5()
tsum10 = clim_reg.getTemperatureSum10()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
#======================
# plt.figure(1, figsize=(20, 4),dpi=300)
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(tsum0, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 0')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 2)
plt.imshow(tsum5, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 5')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 3)
plt.imshow(tsum10, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 10')
plt.colorbar(shrink=0.8)
#----------------------
plt.savefig(out_path+"Tsum_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
#======================

In [ ]:
import psutil
print('1:',psutil.virtual_memory().free/1E9)


In [ ]:
# parallel=True; for global expect ~s (laptop), ~4.75min (desktop) run time 
# parallel=True; for china expect x (laptop), ~8s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time
start=timer()

tprofile = clim_reg.getTemperatureProfile()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
tile_list = ['A1','A2','A3','A4','A5','A6','A7','A8','A9',
            'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']

# fig = plt.figure(figsize=(12, 12))
fig = plt.figure()
for i1 in range(1, 19):
    plt.subplot(6, 3, i1)
    plt.imshow(tprofile[i1-1],interpolation='none')
    plt.title(tile_list[i1-1])
    plt.colorbar(shrink=0.8)
plt.tight_layout()

plt.savefig(out_path+"Tprofiles_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
def Eta_class(mask,ta,tx,tmelt,lgpt5):    

    eta_class=np.empty(mask.shape,dtype='float32')
    eta_class[:]=np.nan

    # assign categorical value to snow areas
    eta_class=np.where((ta<=0) & (tx<=tmelt),np.float32(1),np.float32(eta_class))

    # assign categorical value to snow melting areas
    eta_class=np.where((ta<=0) & (tx>=0) & ~np.isfinite(eta_class),np.float32(2),np.float32(eta_class))

    # assign categorical value to cold areas (pre-growing season)
    eta_class=np.where((ta>0) & (ta<5) & ~np.isfinite(eta_class),np.float32(3),np.float32(eta_class))

    # assign categorical value to warm areas (growing season)
    eta_class=np.where((lgpt5<365) & (ta>=5) & ~np.isfinite(eta_class),np.float32(4),np.float32(eta_class))

    # assign categorical value to warmest areas
    eta_class=np.where((mask==1) & ~np.isfinite(eta_class),np.float32(5),np.float32(eta_class))

In [ ]:
def Eta_class(mask,lgpt5,ta,tx,tmelt):    

    eta_class=da.empty(mask.shape,dtype='float32')
    eta_class[:]=np.nan

    eta_class=da.where((ta<=0) & (tx<=tmelt),np.float32(1),np.float32(eta_class))
    eta_class=da.where((ta<=0) & (tx>=0) & ~np.isfinite(eta_class),np.float32(2),np.float32(eta_class))
    eta_class=da.where((ta>0) & (ta<5) & ~np.isfinite(eta_class),np.float32(3),np.float32(eta_class))
    eta_class=da.where((lgpt5<365) & (ta>=5) & ~np.isfinite(eta_class),np.float32(4),np.float32(eta_class))
    eta_class=da.where((mask==1) & ~np.isfinite(eta_class),np.float32(5),np.float32(eta_class))

    return eta_class

In [ ]:
start=timer()
print('in ClimateRegime, computing Tx365 in parallel')
Tx365=clim_reg.maxT_daily.compute()  # convert dask to numpy
print('in ClimateRegime, computing Ta365 in parallel')
Ta365=clim_reg.meanT_daily.compute()  # convert dask to numpy
print('in ClimateRegime, computing Pcp355 in parallel')
Pcp365=clim_reg.totalPrec_daily.compute()  # convert dask to numpy   
print('computing mask in parallel')
mask=clim_reg.im_mask.compute()

eta_class=Eta_class(im_mask,lgpt5,Ta,Tx,Txsnm)
task_time=timer()-start
task_time

In [ ]:
test=clim_reg.im_mask.to_delayed()
chunk_shapes2D=dask.compute([chunk.shape for chunk in test.ravel()])
# chunk_shapes2D[0][:]

chunk_shapes3D=[]
for item in chunk_shapes2D[0]:
    chunk_shapes3D.append((item[0],item[1],365))
# chunk_shapes3D[:]

In [ ]:
test=clim_reg.meanT_daily.rechunk(chunks=(-1,-1,1))
test

In [ ]:
test2=clim_reg.maxT_daily.rechunk(chunks=(-1,-1,1))
test2

In [ ]:
Txsnm = 0.
eta_class=Eta_class(clim_reg.im_mask,da.from_array(lgpt5,chunks=clim_reg.chunk2D),clim_reg.meanT_daily[:,:,0],clim_reg.maxT_daily[:,:,0],Txsnm)
eta_class

In [ ]:
# parallel=True; for global expect ~s (laptop), ~x (desktop) run time 
# parallel=True; for china expect x (laptop), ~52s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time
start=timer()

lgp = clim_reg.getLGP(Sa=100., D=1.)
lgp_class = clim_reg.getLGPClassified(lgp)
lgp_equv = clim_reg.getLGPEquivalent()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
# fig = plt.figure(figsize=(20, 4))
fig = plt.figure()
# fig = plt.figure(figsize=(10, 8),dpi=600)
plt.subplot(121)
plt.imshow(lgp, cmap='viridis', vmin=0, vmax=366,interpolation='none')
plt.title('LGP [days]')
plt.colorbar(shrink=0.8)
# plt.savefig(out_path+"LGP_"+revname+".png", bbox_inches="tight", dpi=600)
# plt.show()

# fig = plt.figure(figsize=(20, 8),dpi=600)
plt.subplot(122)
plt.imshow(lgp_equv, cmap='viridis', vmin=0, vmax=366,interpolation='none')
plt.title('LGP Equivalent [days]')
plt.colorbar(shrink=0.8)
# plt.savefig(out_path+"LGP_Equv_"+revname+".png",bbox_inches="tight", dpi=600)
plt.show()

plt.savefig(out_path+"LGP_and_LGP_Equv_"+revname+".png",bbox_inches="tight", dpi=300)

In [ ]:
# parallel=True; for global expect ~s (laptop), ~x (desktop) run time 
# parallel=True; for china expect x (laptop), ~7s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time

# expect >1s run time for china
# non-vectorized: expect ~1.5 minutes run time for china
start=timer()

multi_crop = clim_reg.getMultiCroppingZones(tclimate, lgp, lgpt5, lgpt10, tsum0, tsum10)
multi_crop_rainfed = multi_crop[0]  # for rainfed conditions
multi_crop_irr = multi_crop[1]  # for irrigated conditions

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(multi_crop_irr, cmap=plt.get_cmap('gist_ncar_r', 9), vmin=-0.2, vmax=8.4,interpolation='none')
plt.title('Multi Cropping Zone - IRR')
plt.colorbar()
plt.savefig(out_path+"multicrop_irr_"+revname+".png", bbox_inches="tight", dpi=300)
plt.show()

fig = plt.figure()
plt.imshow(multi_crop_rainfed,cmap=plt.get_cmap('gist_ncar_r', 9), vmin=-0.2, vmax=8.4,interpolation='none')
plt.title('Multi Cropping Zone - RAINFED')
plt.colorbar()
plt.savefig(out_path+"multicrop_rain_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
# parallel=True; for global expect ~s (laptop), ~x (desktop) run time 
# parallel=True; for china expect x (laptop), ~1s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time

# expect >1s run time for china
# non-vectorized: expect ~2 seconds run time for china
start=timer()

permafrost_eval = clim_reg.AirFrostIndexandPermafrostEvaluation()
frost_index = permafrost_eval[0]
permafrost = permafrost_eval[1]

task_time=timer()-start
task_time

In [ ]:
frost_index.dtype

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(frost_index, cmap=plt.get_cmap('tab20b', 11), vmin=-0.05, vmax=1.05,interpolation='none')
plt.title('Frost Index')
plt.colorbar()
plt.savefig(out_path+"frost_index_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

fig = plt.figure()
plt.imshow(permafrost, cmap=plt.get_cmap('tab20b', 5), vmin=-0.5, vmax=4.3,interpolation='none')
plt.title('Permafrost Evaluation')
plt.colorbar()
plt.savefig(out_path+"permafrost_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

In [ ]:
# parallel=True; for global expect ~s (laptop), ~x (desktop) run time 
# parallel=True; for china expect x (laptop), <1s (desktop) run time
# parallel=False; for china expect x (laptop), ~xmin (desktop) run time

# expect >1s run time for china
# expect ~15 seconds run time for china
start=timer()

tzone_fallow = clim_reg.TZoneFallowRequirement(tzone)

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(tzone_fallow, cmap=plt.get_cmap('tab10', 7), vmin=-0.5, vmax=6.3,interpolation='none')
plt.title('Fallow Requirement')
plt.colorbar()
plt.savefig(out_path+"fallow_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'fallow_'+revname+'.tif', tzone_fallow)


In [ ]:
clim_reg.__dict__.keys()

In [ ]:
# check dtypes of all outputs
outputs=[tclimate,tzone,lgpt0,lgpt5,lgpt10,tsum0,tsum5,tsum10,tprofile,lgp,lgp_class,lgp_equv,multi_crop_rainfed,multi_crop_irr,frost_index,permafrost,tzone_fallow]
dtypes=[]
for var in outputs:
    # for arrays
    try:
        dt=var.dtype
        dtypes.append(dt)
    # for lists of arrays
    except:
        for v in var:
            dt=v.dtype
            dtypes.append(dt)

print(dtypes)

In [ ]:
vars=['meanT_daily','meanT_daily_sealevel','P_by_PET_daily','maxT_daily','totalPrec_daily']

for v in vars:

    start=timer()
    test=clim_reg.__getattribute__(v).compute()
    task_time=timer()-start
    print(v, task_time, 'seconds')

In [ ]:
def ETtest(dstart,dend,cdims,lat,alt,tmn,tmx,u2m,srad,rh):
    # print('implementing data value limits')
    nlats=cdims[0]
    nlons=cdims[1]

    # print('tavg,lam,dayoyr,ndays')
    tavg = 0.5*(tmn + tmx)  # Averaged temperature  #KLG
    lam = 2.501 - 0.002361 * tavg  # Latent heat of vaporization
    dayoyr = np.arange(dstart, dend+1)  # Julien Days #KLG
    ndays=len(dayoyr)  #KLG            

    # print('es_tmin,es_tmax,es,ea')
    es_tmin = 0.6108 * np.exp((17.27 * tmn) / (tmn + 237.3))
    es_tmax = 0.6108 * np.exp((17.27 * tmx) / (tmx + 237.3))
    es = 0.5*(es_tmin + es_tmax)
    ea = rh * es  # Actual Vapor Pressure derived from relative humidity

    # print('dlmx,dlmn,dl')
    dlmx = 4098. * es_tmax / (tmx + 237.3)**2
    dlmn = 4098. * es_tmin / (tmn + 237.3)**2
    del es_tmin, es_tmax  #KLG
    dl = 0.5* (dlmx + dlmn)
    del dlmx,dlmn  #KLG

    # print('ap and constants')
    ap = 101.3*np.power(((293-(0.0065*alt))/293), 5.256)
    ap = np.tile(ap[:,:,np.newaxis],(1,1,ndays))  
    gam = 0.0016286 * ap/lam
    hw = 200.
    ht = 190.
    hc = 12
    Rl = 100  
    RLAI = 24 * 0.12
    rhoc = Rl/(0.5*RLAI)  
    del ap

    # print('rhoa,gamst')
    rhoa = 208/u2m
    gamst = gam * (1. + rhoc/rhoa)

    # print('lat,sdcl')
    latr = lat * np.pi/180.
    latr3D = np.tile(latr[:,:,np.newaxis],(1,1,ndays))
    del latr
    sdcl = 0.4093 * np.sin(0.017214206 * dayoyr - 1.405)
    sdcl3D = np.tile(sdcl[np.newaxis, np.newaxis,:], (nlats,nlons,1))
    del sdcl

    # print('xx,yy,zz')
    xx = np.sin(sdcl3D) * np.sin(latr3D)
    yy = np.cos(sdcl3D) * np.cos(latr3D)
    zz = xx/yy

    # print('omg,dayhr')
    with np.errstate(invalid='ignore'):
        omg = np.tan(zz / (1. - zz*zz)**0.5) + 1.5708
    dayhr = 24. * (omg/np.pi)

    omg=np.where((np.abs(zz)>=0.9999)&(zz>0),np.pi,omg)
    dayhr=np.where((np.abs(zz) >= 0.9999)&(zz > 0), 23.999,dayhr)
    omg=np.where((np.abs(zz) >= 0.9999)&(zz <= 0),0,omg)
    dayhr=np.where((np.abs(zz) >= 0.9999)&(zz <= 0), 0.001,dayhr)
    del latr3D,sdcl3D,zz

    # print('sdst,ra')
    sdst = 1.0 + 0.033 * np.cos(0.017214206 * dayoyr)
    sdst3D = np.tile(sdst[np.newaxis, np.newaxis,:], (nlats,nlons,1))
    del sdst
    ra = 37.586 * sdst3D * (omg*xx + np.sin(omg)*yy)
    # ra = 37.586 * sdst3D * (omg_rev2*xx + np.sin(omg_rev2)*yy)
    # del sdst3D, sdst, omg, omg_rev1, omg_rev2, dayhr, dayhr_rev1, dayhr_rev2, xx, yy       
    del sdst3D, omg, dayhr, xx, yy       

    # print('rs,rs0')
    rs = srad  
    alt3D=np.tile(alt[:,:,np.newaxis],(1,1,ndays)) 
    del alt
    rs0 = (0.75 + 0.00002 * alt3D) * ra
    del alt3D

    # print('rns,rnl')
    rns = 0.77 * rs
    sub_cst = 0.000000004903
    with np.errstate(invalid='ignore',divide='ignore'):
        rs_div_ds0=rs/rs0
    rnl = (((273.16+tmx)**4)+((273.16 + tmn)**4)) * \
        (0.34 - (0.14*(ea**0.5))) * \
        ((1.35*(rs_div_ds0))-0.35)*sub_cst/2
    del rs0,rs,rs_div_ds0,tmx,tmn

    # print('rn')
    rn = rns - rnl
    del rns,rnl

    # print('ta_diff,G')
    ta_diff=np.diff(tavg,n=1,axis=2,prepend=tavg[:,:,-1:])  #KLG
    G = 0.14 * ta_diff  #KLG
    del ta_diff

    # print('et0ady')
    et0ady = gam/(dl+gamst) * 900./(tavg+273.) * u2m * (es-ea)
    del gam,tavg,es,ea

    # print('et0rad')
    et0rad = dl/(dl+gamst) * (rn-G) / lam
    del dl,gamst,rn,G,lam

    # print('et0')
    et0 = et0ady + et0rad
    del et0ady,et0rad

    # print('et0_out')
    et0_out=np.where(et0<0,0,et0)

    return et0_out
    # return doy_start,doy_end,chunkdims,latitude,alt,min_temp,max_temp,u2m,shortrad_daily_MJm2day,rel_humidity
    # return alt

In [ ]:
def setDailyClimateData(self, min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity):

    self.doy_start=1  #KLG
    self.doy_end=min_temp.shape[2]  #KLG 
    
    print('rechunking and delaying inputs')
    min_temp=min_temp.rechunk(chunks=self.chunk3D)
    max_temp=max_temp.rechunk(chunks=self.chunk3D)
    tmn_delay=min_temp.to_delayed()
    tmx_delay=max_temp.to_delayed()

    short_rad=short_rad.rechunk(chunks=self.chunk3D) # chunk
    short_rad=da.where(short_rad < 0, 0, short_rad)  # elim negatives
    short_rad = (short_rad*3600.*24.)/1000000.       # convert units
    srad_delay=short_rad.to_delayed()
    del short_rad

    wind_speed=wind_speed.rechunk(chunks=self.chunk3D)     # chunk
    # wind_speed=da.where(wind_speed < 0, 0, wind_speed)     # elim negative
    wind_speed=da.where(wind_speed < 0.5, 0.5, wind_speed) # elim negative and small values
    wind_delay=wind_speed.to_delayed()

    rel_humidity=rel_humidity.rechunk(chunks=self.chunk3D)        # chunk
    rel_humidity=da.where(rel_humidity > 0.99, 0.99,rel_humidity) # elim high values
    rel_humidity=da.where(rel_humidity < 0.05, 0.05,rel_humidity) # elim low values
    rh_delay=rel_humidity.to_delayed()
    del rel_humidity

    lat_delay=self.latitude.to_delayed()
    elev_delay=self.elevation.to_delayed()        


    print('getting chunk shapes')
    chunk_shapes=dask.compute([chunk.shape for chunk in tmn_delay.ravel()])
    print('zipping delayed inputs')
    zipvars=zip(chunk_shapes[0][:],lat_delay.ravel(),elev_delay.ravel(),tmn_delay.ravel(),tmx_delay.ravel(),wind_delay.ravel(),srad_delay.ravel(),rh_delay.ravel())
    print('creating task list')
    task_list=[dask.delayed(ETtest)(self.doy_start,self.doy_end,cshape,lat,el,tmn,tmx,u,srad,rh) for cshape,lat,el,tmn,tmx,u,srad,rh in zipvars]

    print('COMPUTING')
    result_chunks=dask.compute(*task_list)
    print('CONCATENATING')
    self.pet_daily=np.concatenate(result_chunks,axis=1)
    # return result_chunks


    print('computing meanT, meanT_daily_sealevel, P_by_PET_daily')
    self.meanT_daily = 0.5*(min_temp+max_temp)  #KLG

    # sea level temperature
    # P over PET ratio (to eliminate nan in the result, nan is replaced with zero)
    if self.parallel:
        self.meanT_daily_sealevel = self.meanT_daily + (da.tile(self.elevation[:,:,np.newaxis],(1,1,self.doy_end)).rechunk(chunks=self.chunk3D)/100*0.55)
        precipitation=precipitation.rechunk(chunks=self.chunk3D)
        with np.errstate(invalid='ignore',divide='ignore'):
            self.P_by_PET_daily = da.nan_to_num(precipitation / self.pet_daily)
    else:
        self.meanT_daily_sealevel = self.meanT_daily + np.expand_dims(self.elevation/100*0.55,axis=2) # automatic broadcasting #KLG        
        with np.errstate(invalid='ignore',divide='ignore'):
            self.P_by_PET_daily = np.nan_to_num(precipitation / self.pet_daily)

    self.set_monthly = False

    print('computing interp_daily_temp')
    # smoothed daily temperature
    # create a mask of all 1's if the user doesn't provide a mask
    if self.set_mask:
        mask=self.im_mask
    else:
        mask=np.ones((self.im_height,self.im_width),dtype='int')

    obj_utilities = UtilitiesCalc.UtilitiesCalc(self.chunk2D,self.chunk3D)  #KLG
    self.interp_daily_temp=obj_utilities.smoothDailyTemp(self.doy_start,self.doy_end, mask, self.meanT_daily,self.chunk3D)

    print('copying maxT_daily,minT_daily,totalPrec_daily')
    self.maxT_daily = max_temp
    # self.minT_daily = min_temp
    self.totalPrec_daily = precipitation  #KLG   

In [ ]:
clim_reg.chunk3D, clim_reg.chunksize3D_MB

In [ ]:
setDailyClimateData(clim_reg, min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

In [ ]:
# clim_reg.pet_daily # numpy array
# clim_reg.meanT_daily  # dask array
# clim_reg.meanT_daily_sealevel  # dask_array
# clim_reg.P_by_PET_daily  # dask array
# clim_reg.maxT_daily  # dask array
# clim_reg.totalPrec_daily  # dask array
# clim_reg.interp_daily_temp  # numpy array

In [ ]:
prsum=clim_reg.totalPrec_daily.sum(axis=2).compute()

In [ ]:
clim_reg.P_by_PET_daily.compute()

In [ ]:
# this normal numpy calculation works just fine with 5GB inputs

print('implementing data value limits')
doy_start=1  #KLG
doy_end=min_temp.shape[2]  #KLG
nlats=min_temp.shape[0]
nlons=min_temp.shape[1]

rel_humidity=np.where(rel_humidity > 0.99, 0.99,rel_humidity)
rel_humidity=np.where(rel_humidity < 0.05, 0.05,rel_humidity)
short_rad=np.where(short_rad < 0, 0, short_rad)
shortrad_daily_MJm2day = (short_rad*3600.*24.)/1000000.
del short_rad
# wind_speed=da.where(wind_speed < 0, 0, wind_speed)  
u2m=np.where(wind_speed < 0.5, 0.5, wind_speed)

print('tavg,lam,dayoyr,ndays')
tavg = 0.5*(min_temp + max_temp)  # Averaged temperature  #KLG
lam = 2.501 - 0.002361 * tavg  # Latent heat of vaporization

dayoyr = np.arange(doy_start, doy_end+1)  # Julien Days #KLG
ndays=len(dayoyr)  #KLG            
# alt=np.tile(clim_reg.elevation[:,:,np.newaxis],(1,1,ndays))  # 3D altitude #KLG
alt=elevation

print('es_tmin,es_tmax,es,ea')
es_tmin = 0.6108 * np.exp((17.27 * min_temp) / (min_temp + 237.3))
es_tmax = 0.6108 * np.exp((17.27 * max_temp) / (max_temp + 237.3))
es = 0.5*(es_tmin + es_tmax)
ea = rel_humidity * es  # Actual Vapor Pressure derived from relative humidity

print('dlmx,dlmn,dl')
dlmx = 4098. * es_tmax / (max_temp + 237.3)**2
dlmn = 4098. * es_tmin / (min_temp + 237.3)**2
# del es_tmin, es_tmax  #KLG
dl = 0.5* (dlmx + dlmn)
# del dlmx,dlmn  #KLG

print('ap and constants')
ap = 101.3*np.power(((293-(0.0065*alt))/293), 5.256)
ap = np.tile(ap[:,:,np.newaxis],(1,1,ndays))  
gam = 0.0016286 * ap/lam
hw = 200.
ht = 190.
hc = 12
Rl = 100  
RLAI = 24 * 0.12
rhoc = Rl/(0.5*RLAI)  
# del ap

print('rhoa,gamst')
rhoa = 208/u2m
gamst = gam * (1. + rhoc/rhoa)

print('lat,sdcl')
latr = clim_reg.latitude * np.pi/180.
latr3D = np.tile(latr[:,:,np.newaxis],(1,1,ndays))
sdcl = 0.4093 * np.sin(0.017214206 * dayoyr - 1.405)
sdcl3D = np.tile(sdcl[np.newaxis, np.newaxis,:], (nlats,nlons,1))

print('xx,yy,zz')
xx = np.sin(sdcl3D) * np.sin(latr3D)
yy = np.cos(sdcl3D) * np.cos(latr3D)
zz = xx/yy

print('omg,dayhr')
omg = np.tan(zz / (1. - zz*zz)**0.5) + 1.5708
dayhr = 24. * (omg/np.pi)
omg_rev1=np.where((np.abs(zz)>=0.9999)&(zz>0),np.pi,omg)
dayhr_rev1=np.where((np.abs(zz) >= 0.9999)&(zz > 0), 23.999,dayhr)
omg_rev2=np.where((np.abs(zz) >= 0.9999)&(zz <= 0),0,omg_rev1)
dayhr_rev2=np.where((np.abs(zz) >= 0.9999)&(zz <= 0), 0.001,dayhr_rev1)
del latr3D,sdcl3D,latr,sdcl,zz

print('sdst,ra')
sdst = 1.0 + 0.033 * np.cos(0.017214206 * dayoyr)
sdst3D = np.tile(sdst[np.newaxis, np.newaxis,:], (nlats,nlons,1))
ra = 37.586 * sdst3D * (omg_rev2*xx + np.sin(omg_rev2)*yy)
del sdst3D, sdst, omg, omg_rev1, omg_rev2, dayhr, dayhr_rev1, dayhr_rev2, xx, yy       

print('rs,rs0')
rs = shortrad_daily_MJm2day  
alt3D=np.tile(alt[:,:,np.newaxis],(1,1,ndays)) 
rs0 = (0.75 + 0.00002 * alt3D) * ra
del alt3D,alt

print('rns,rnl')
rns = 0.77 * rs
sub_cst = 0.000000004903
rs_div_ds0=rs/rs0
rnl = (((273.16+max_temp)**4)+((273.16 + min_temp)**4)) * \
    (0.34 - (0.14*(ea**0.5))) * \
    ((1.35*(rs_div_ds0))-0.35)*sub_cst/2
del rs0,rs,rs_div_ds0

print('rn')
rn = rns - rnl
del rns,rnl

print('ta_diff,G')
ta_diff=np.diff(tavg,n=1,axis=2,prepend=tavg[:,:,-1:])  #KLG
G = 0.14 * ta_diff  #KLG
del ta_diff

print('et0ady')
et0ady = gam/(dl+gamst) * 900./(tavg+273.) * u2m * (es-ea)
del gam,tavg,es,ea

print('et0rad')
et0rad = dl/(dl+gamst) * (rn-G) / lam
del dl,gamst,rn,G,lam

print('et0')
et0 = et0ady + et0rad
del et0ady,et0rad

print('et0_out')
et0_out=np.where(et0<0,0,et0)

In [ ]:
gam.shape, rhoa.shape

In [ ]:
# gamst

In [ ]:
# nchunks=238
# nlons=min_temp.shape[1]
# chunk_nlons=int(da.ceil(nlons/nchunks))
# chunks3D=(-1,chunk_nlons,-1)
# chunks2D=(-1,chunk_nlons)

# lat_vals=da.linspace(lat_min, lat_max, min_temp.shape[0]).astype('float32')  #KLG
# lat_map=da.tile(lat_vals[:,np.newaxis],(1,min_temp.shape[1])).rechunk(chunks=chunks2D)  #KLG   
# # lat_map=da.broadcast_to(lat_vals,(min_temp.shape[0],min_temp.shape[1]),chunks=chunks2D)
# lat_map

In [ ]:
class ETOCalc(object):
 
    def __init__(self, cycle_begin, cycle_end, latitude, altitude,chunk3D):
        """Initiate a ETOCalc Class instance

        Args:
            cycle_begin (int): Julian day for the beginning of crop cycle
            cycle_end (int): Julian day for the ending of crop cycle
            latitude (float): a latitude value
            altitude (float): an altitude value
        """        
        self.cycle_begin = cycle_begin
        self.cycle_end = cycle_end
        self.latitude = latitude
        self.alt = altitude
        
        self.chunk3D=chunk3D
        if self.chunk3D: self.parallel=True
        else: self.parallel=False
        print('parallel ETO = ',self.parallel)

    def setClimateData(self, min_temp, max_temp, wind_speed, short_rad, rel_humidity):
        """Load the climatic (point) data into the Class

        Args:
            min_temp (float): Minimum temperature [Celcius]
            max_temp (float): Maximum temperature [Celcius]
            wind_speed (float): Windspeed at 2m altitude [m/s]
            short_rad (float): Radiation [MJ/m2.day]
            rel_humidity (float): Relative humidity [decimal percentage]
        """        

        self.minT_daily = min_temp # Celcius
        self.maxT_daily = max_temp # Celcius
        self.u2m = wind_speed # m/s at 2m
        self.shortRad_daily = short_rad # MJ/m2.day
        self.rel_humidity = rel_humidity # Fraction

    def calculateETO(self):  #KLG
        # numba doesn't speed this up in time tests  #KLG
        # removing in favor of vectorization which will allow chunking with dask for speed  #KLG

        """Calculate the reference evapotranspiration with Penmann-Monteith Equation

        Returns:
            ## float: ETo of a single pixel (function is called pixel-wise)
            float: ETo of each pixel  #KLG
        """        
        # if self.parallel:
        #     self.minT_daily=self.minT_daily.persist()
        #     self.maxT_daily=self.maxT_daily.persist()
            # self.windspeed_daily=self.windspeed_daily.persist()
            # self.alt=self.alt.persist()
            # self.latitude=self.latitude.persist()

        # nchunks=238
        # nlons=self.alt.shape[1]
        # chunk_nlons=int(da.ceil(nlons/nchunks))
        # chunks3D=(-1,chunk_nlons,-1)
        # chunks2D=(-1,chunk_nlons)

        print('tavg,lam,alt')
        # constants
        # tavg = 0.5*(maxT_daily+minT_daily)  # Averaged temperature
        tavg = 0.5*(self.minT_daily + self.maxT_daily)  # Averaged temperature  #KLG
        print('tavg.shape=',tavg.shape)
        lam = 2.501 - 0.002361 * tavg  # Latent heat of vaporization
        # if self.parallel:
        #     dayoyr = da.arange(self.cycle_begin, self.cycle_end+1,chunks=-1)  # Julien Days #KLG
        #     ndays=len(dayoyr)  #KLG
        #     alt=da.tile(self.alt[:,:,np.newaxis],(1,1,ndays)).rechunk(chunks=self.chunk3D)
        # else:
            # dayoyr = np.arange(self.cycle_begin, self.cycle_end+1)  # Julien Days #KLG
            # ndays=len(dayoyr)  #KLG            
            # alt=np.tile(self.alt[:,:,np.newaxis],(1,1,ndays))  # 3D altitude #KLG
        dayoyr = np.arange(self.cycle_begin, self.cycle_end+1)  # Julien Days #KLG
        ndays=len(dayoyr)  #KLG            
        alt=np.tile(self.alt[:,:,np.newaxis],(1,1,ndays))  # 3D altitude #KLG

        # print('u2m')
        # Wind speed
        # u2m = windspeed_daily.copy()
        # u2m = self.windspeed_daily.copy()  #KLG
        # limit to no less than 0.5 m/s; FAO 56, p.63
        # u2m[windspeed_daily < 0.5] = 0.5
        # print('u2m.where')
        # if self.parallel:
        #     u2m=da.where(u2m < 0.5, 0.5, u2m)
        # else:
        #     u2m=np.where(u2m < 0.5, 0.5, u2m)
        # u2m=np.where(u2m < 0.5, 0.5, u2m)

        print('es_tmin,es_tmax')
        # Mean Saturation Vapor Pressure derived from air temperature
        # if self.parallel:
        #     es_tmin = 0.6108 * da.exp((17.27 * self.minT_daily) / (self.minT_daily + 237.3))  #KLG
        #     es_tmax = 0.6108 * da.exp((17.27 * self.maxT_daily) / (self.maxT_daily + 237.3))  #KLG
        # else:            
        #     es_tmin = 0.6108 * np.exp((17.27 * self.minT_daily) / (self.minT_daily + 237.3))
        #     es_tmax = 0.6108 * np.exp((17.27 * self.maxT_daily) / (self.maxT_daily + 237.3))
        es_tmin = 0.6108 * np.exp((17.27 * self.minT_daily) / (self.minT_daily + 237.3))
        es_tmax = 0.6108 * np.exp((17.27 * self.maxT_daily) / (self.maxT_daily + 237.3))

        print('es,ea')
        es = 0.5*(es_tmin + es_tmax)
        # ea = rel_humidity * es  # Actual Vapor Pressure derived from relative humidity
        ea = self.rel_humidity * es  # Actual Vapor Pressure derived from relative humidity  #KLG

        print('dlmx,dlmn,dl')
        # slope vapour pressure curve
        # dlmx = 4098. * es_tmax / (maxT_daily + 237.3)**2
        # dlmn = 4098. * es_tmin / (minT_daily + 237.3)**2
        dlmx = 4098. * es_tmax / (self.maxT_daily + 237.3)**2  #KLG
        dlmn = 4098. * es_tmin / (self.minT_daily + 237.3)**2  #KLG
        del es_tmin, es_tmax  #KLG
        dl = 0.5* (dlmx + dlmn)
        del dlmx,dlmn  #KLG

        print('ap,gam')
        # Atmospheric pressure
        # if self.parallel:
        #     ap = 101.3*da.power(((293-(0.0065*alt))/293), 5.256)
        # else:
        #     ap = 101.3*np.power(((293-(0.0065*alt))/293), 5.256)
        ap = 101.3*np.power(((293-(0.0065*alt))/293), 5.256)

            
        # Psychrometric constant
        gam = 0.0016286 * ap/lam
        del ap

        hw = 200.
        ht = 190.
        hc = 12.

        print('rhoa,Rl,RLAI,rhic,gamst')
        # aerodynamic resistance
        rhoa = 208/self.u2m

        # crop canopy resistance
        Rl = 100  # daily stomata resistance of a single leaf (s/m)
        # Standard is xLAO = 24
        RLAI = 24 * 0.12
        rhoc = Rl/(0.5*RLAI)  # crop canopy resistance

        gamst = gam * (1. + rhoc/rhoa)

        print('latr')
        # latr = latitude * np.pi/180.
        latr = self.latitude * np.pi/180.
        # if self.parallel:
        #     latr=da.tile(latr[:,:,np.newaxis],(1,1,ndays)).rechunk(chunks=self.chunk3D)
        # else:
        #     latr = np.tile(latr[:,:,np.newaxis],(1,1,ndays))
        # latr = np.tile(latr[:,:,np.newaxis],(1,1,ndays))

        print('sdcl')
        # (a) calculate extraterrestrial radiation
        # solar declination (rad)
        # if self.parallel:
        #     sdcl = 0.4093 * da.sin(0.017214206 * dayoyr - 1.405)
        #     sdcl=da.broadcast_to(sdcl,(tavg.shape[0],tavg.shape[1],tavg.shape[2]),chunks=self.chunk3D)
        # else:
        #     sdcl = 0.4093 * np.sin(0.017214206 * dayoyr - 1.405)
        #     sdcl = np.tile(sdcl[np.newaxis, np.newaxis,:], (tavg.shape[0],tavg.shape[1],1))
        sdcl = 0.4093 * np.sin(0.017214206 * dayoyr - 1.405)
        # sdcl = np.tile(sdcl[np.newaxis, np.newaxis,:], (tavg.shape[0],tavg.shape[1],1))
        
        print('sdst')
        # relative distance earth to sun
        # if self.parallel:
        #     sdst = 1.0 + 0.033 * da.cos(0.017214206 * dayoyr)
        #     sdst=da.broadcast_to(sdst,(tavg.shape[0],tavg.shape[1],tavg.shape[2]),chunks=self.chunk3D)
        # else:
        #     sdst = 1.0 + 0.033 * np.cos(0.017214206 * dayoyr)
        #     sdst=np.tile(sdst[np.newaxis, np.newaxis,:], (tavg.shape[0],tavg.shape[1],1))
        sdst = 1.0 + 0.033 * np.cos(0.017214206 * dayoyr)
        # sdst=np.tile(sdst[np.newaxis, np.newaxis,:], (tavg.shape[0],tavg.shape[1],1))
        del dayoyr

        print('xx,yy,zz')
        # if self.parallel:
        #     xx = da.sin(sdcl) * da.sin(latr)
        #     yy = da.cos(sdcl) * da.cos(latr)            
        # else:
        #     xx = np.sin(sdcl) * np.sin(latr)
        #     yy = np.cos(sdcl) * np.cos(latr)
        xx = np.sin(sdcl) * np.sin(latr)
        yy = np.cos(sdcl) * np.cos(latr)
        zz = xx/yy

        print('omg,dayhr')
        # if self.parallel:
        #     omg = da.tan(zz / (1. - zz*zz)**0.5) + 1.5708
        # else:
        #     omg = np.tan(zz / (1. - zz*zz)**0.5) + 1.5708
        omg = np.tan(zz / (1. - zz*zz)**0.5) + 1.5708
        dayhr = 24. * (omg/np.pi)

        # if self.parallel:
        #     print('omg.where,dayhr.where,ra')
        #     omg=da.where((da.abs(zz)>=0.9999)&(zz>0),np.pi,omg)
        #     dayhr=da.where((da.abs(zz) >= 0.9999)&(zz > 0), 23.999,dayhr)

        #     omg=da.where((da.abs(zz) >= 0.9999)&(zz <= 0),0,omg)
        #     dayhr=da.where((da.abs(zz) >= 0.9999)&(zz <= 0), 0.001,dayhr)

        #     print('ra')
        #     ra = 37.586 * sdst * (omg*xx + da.sin(omg)*yy)
        # else:
        #     print('omg.where,dayhr.where,ra')
        #     omg=np.where((np.abs(zz)>=0.9999)&(zz>0),np.pi,omg)
        #     dayhr=np.where((np.abs(zz) >= 0.9999)&(zz > 0), 23.999,dayhr)

        #     omg=np.where((np.abs(zz) >= 0.9999)&(zz <= 0),0,omg)
        #     dayhr=np.where((np.abs(zz) >= 0.9999)&(zz <= 0), 0.001,dayhr) 
            
        #     print('ra')
        #     ra = 37.586 * sdst * (omg*xx + np.sin(omg)*yy)
        print('omg.where,dayhr.where,ra')
        omg_rev1=np.where((np.abs(zz)>=0.9999)&(zz>0),np.pi,omg)
        dayhr_rev1=np.where((np.abs(zz) >= 0.9999)&(zz > 0), 23.999,dayhr)

        omg_rev2=np.where((np.abs(zz) >= 0.9999)&(zz <= 0),0,omg_rev1)
        dayhr_rev2=np.where((np.abs(zz) >= 0.9999)&(zz <= 0), 0.001,dayhr_rev1) 
        
        print('ra')
        ra = 37.586 * sdst * (omg_rev2*xx + np.sin(omg_rev2)*yy)
        del sdcl, latr, sdst, omg, omg_rev1, omg_rev2, dayhr, dayhr_rev1, dayhr_rev2, xx, yy, zz        

        print('rs,rs0')
        # (b) solar radiation Rs (0.25, 0.50 Angstrom coefficients)
        # rs = (0.25 + (0.50 * (sd/dayhr))) * ra
        # rs = shortRad_daily
        rs = self.shortRad_daily  #KLG
        # rs0 = (0.75 + 0.00002 * alt) * ra
        rs0 = (0.75 + 0.00002 * self.alt) * ra
        
        del alt

        print('rns,rnl')
        # (c) net shortwave radiation Rns = (1 - alpha) * Rs
        # (alpha for grass = 0.23)
        rns = 0.77 * rs

        # (d) net longwave radiation Rnl
        # Stefan-Boltzmann constant [MJ K-4 m-2 day-1]
        sub_cst = 0.000000004903
        rs_div_ds0=rs/rs0
        rnl = (((273.16+self.maxT_daily)**4)+((273.16 + self.minT_daily)**4)) * \
            (0.34 - (0.14*(ea**0.5))) * \
            ((1.35*(rs_div_ds0))-0.35)*sub_cst/2
        del rs0,rs

        print('rn')
        # (e) net radiation Rn = Rns - Rnl
        rn = rns - rnl
        del rns,rnl

        print('ta_diff,G')
        # (f) soil heat flux [MJ/m2/day]
        # if self.parallel:
        #     ta_diff=da.diff(tavg,n=1,axis=2,prepend=tavg[:,:,-1:])  #KLG
        # else:
        #     ta_diff=np.diff(tavg,n=1,axis=2,prepend=tavg[:,:,-1:])  #KLG
        ta_diff=np.diff(tavg,n=1,axis=2,prepend=tavg[:,:,-1:])  #KLG
        G = 0.14 * ta_diff  #KLG
        del ta_diff

        print('et0ady')
        # (g) calculate aerodynamic and radiation terms of ET0
        et0ady = gam/(dl+gamst) * 900./(tavg+273.) * self.u2m * (es-ea)
        del gam,tavg,es,ea

        print('et0rad')
        et0rad = dl/(dl+gamst) * (rn-G) / lam
        del dl,gamst,rn,G,lam

        print('et0')
        et0 = et0ady + et0rad
        del et0ady,et0rad

        print('et0.where')
        # et0[et0 < 0] = 0
        # if self.parallel:
        #     et0=da.where(et0<0,0,et0)
        # else:
        #     et0=np.where(et0<0,0,et0)
        et0=np.where(et0<0,0,et0)

        return et0

In [ ]:
def compute_pet(doy_start, doy_end, chunk3D, latitude, elevation, min_temp, max_temp, wind_speed, short_rad, rel_humidity):
    print('initializing obj_eto')
    obj_eto = ETOCalc(doy_start, doy_end, latitude, elevation, chunk3D)  #KLG
    
    print('adjusting short_rad units')
    shortrad_daily_MJm2day = (short_rad*3600.*24.)/1000000. # convert w/m2 to MJ/m2/day  #KLG    

    print('obj_eto.setClimateData')
    obj_eto.setClimateData(min_temp, max_temp, wind_speed, shortrad_daily_MJm2day, rel_humidity)  #KLG

    pet_daily= obj_eto.calculateETO()
    return pet_daily


In [ ]:
def setDailyClimateData(self, min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity):

    if self.parallel:
        print('rechunking')
        min_temp=min_temp.rechunk(chunks=self.chunk3D).to_delayed()
        max_temp=max_temp.rechunk(chunks=self.chunk3D).to_delayed()
        # precipitation=precipitation.rechunk(chunks=self.chunk3D)
        short_rad=short_rad.rechunk(chunks=self.chunk3D)
        wind_speed=wind_speed.rechunk(chunks=self.chunk3D)
        rel_humidity=rel_humidity.rechunk(chunks=self.chunk3D)
        # latitude and elevation are already chunked correctly    

        print('removing negative data values')
        rel_humidity=da.where(rel_humidity > 0.99, 0.99,rel_humidity)
        rel_humidity=da.where(rel_humidity < 0.05, 0.05,rel_humidity)
        short_rad=da.where(short_rad < 0, 0, short_rad)
        # wind_speed=da.where(wind_speed < 0, 0, wind_speed)  
        wind_speed=da.where(wind_speed < 0.5, 0.5, wind_speed)

        rel_humidity=rel_humidity.to_delayed()
        short_rad=short_rad.to_delayed()
        wind_speed=wind_speed.to_delayed()

        latitude=self.latitude.to_delayed()
        elevation=self.elevation.to_delayed()        

    self.doy_start=1  #KLG
    self.doy_end=min_temp.shape[2]  #KLG

    task_list=[dask.delayed(compute_pet)(self.doy_start,self.doy_end,self.chunk3D, lat_chunk, elev_chunk, minT_chunk, maxT_chunk, wind_chunk, rad_chunk, rhum_chunk) for lat_chunk, elev_chunk, minT_chunk, maxT_chunk, wind_chunk, rad_chunk, rhum_chunk in zip(latitude[0,:], elevation[0,:], min_temp[0,:], max_temp[0,:], wind_speed[0,:], short_rad[0,:], rel_humidity[0,:])]

    result_chunks=dask.compute(*task_list)
    self.pet_daily=np.concat(result_chunks)

In [ ]:
setDailyClimateData(clim_reg, min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

In [ ]:
def setDailyClimateData(self, min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity):
    # print('converting inputs to dask arrays')
    # nchunks=238
    # nlons=min_temp.shape[1]
    # chunk_nlons=int(np.ceil(nlons/nchunks))
    # chunks3D=(-1,chunk_nlons,-1)
    # chunks2D=(-1,chunk_nlons)
    # print(nchunks,nlons,chunk_nlons,chunks3D)

    # min_temp=da.from_array(min_temp,chunks=chunks3D)
    # max_temp=da.from_array(max_temp,chunks=chunks3D)
    # precipitation=da.from_array(precipitation,chunks=chunks3D)
    # short_rad=da.from_array(short_rad,chunks=chunks3D)
    # wind_speed=da.from_array(wind_speed,chunks=chunks3D)
    # rel_humidity=da.from_array(rel_humidity,chunks=chunks3D)
    # latitude=da.from_array(self.latitude,chunks=chunks2D)
    # elevation=da.from_array(self.elevation,chunks=chunks2D)
    if self.parallel:
        print('rechunking')
        min_temp=min_temp.rechunk(chunks=self.chunk3D)
        max_temp=max_temp.rechunk(chunks=self.chunk3D)
        precipitation=precipitation.rechunk(chunks=self.chunk3D)
        short_rad=short_rad.rechunk(chunks=self.chunk3D)
        wind_speed=wind_speed.rechunk(chunks=self.chunk3D)
        rel_humidity=rel_humidity.rechunk(chunks=self.chunk3D)
        # latitude and elevation are already chunked correctly

        # min_temp=min_temp.persist()
        # max_temp=max_temp.persist()
        # precipitation=precipitation.persist()
        # short_rad=short_rad.persist()
        # wind_speed=wind_speed.persist()
        # rel_humidity=rel_humidity.persist()        

        print('removing negative data values')
        rel_humidity=da.where(rel_humidity > 0.99, 0.99,rel_humidity)
        rel_humidity=da.where(rel_humidity < 0.05, 0.05,rel_humidity)
        short_rad=da.where(short_rad < 0, 0, short_rad)
        wind_speed=da.where(wind_speed < 0, 0, wind_speed)  


    else:
        print('removing negative data values')
        rel_humidity[rel_humidity > 0.99] = 0.99
        rel_humidity[rel_humidity < 0.05] = 0.05
        short_rad[short_rad < 0] = 0
        wind_speed[wind_speed < 0] = 0

    self.doy_start=1  #KLG
    self.doy_end=min_temp.shape[2]  #KLG



    # calculation of reference evapotranspiration (ETo)  #KLG
    print('initializing obj_eto')
    obj_eto = ETOCalc(self.doy_start, self.doy_end, self.latitude, self.elevation,self.chunk3D)  #KLG

    print('adjusting short_rad units')
    shortrad_daily_MJm2day = (short_rad*3600.*24.)/1000000. # convert w/m2 to MJ/m2/day  #KLG
    # shortrad_daily_MJm2day=shortrad_daily_MJm2day.persist()
    print('obj_eto.setClimateData')
    obj_eto.setClimateData(min_temp, max_temp, wind_speed, shortrad_daily_MJm2day, rel_humidity)  #KLG
    del rel_humidity,short_rad,wind_speed,shortrad_daily_MJm2day  #KLG
    # print('pet equations')
    # pet_daily= obj_eto.calculateETO()   #KLG
    # print(pet_daily)
    print('computing pet')
    if self.parallel:
        # self.pet_daily=pet_daily.compute()
        self.pet_daily= obj_eto.calculateETO().compute()   #KLG
        # self.pet_daily= obj_eto.calculateETO()

    else:
        self.pet_daily= obj_eto.calculateETO()
    del obj_eto

    # print('computing meanT, meanT_daily_sealevel, P_by_PET_daily')
    # self.meanT_daily = 0.5*(min_temp+max_temp)  #KLG

    # # sea level temperature
    # # P over PET ratio (to eliminate nan in the result, nan is replaced with zero)
    # if self.parallel:
    #     self.meanT_daily_sealevel = self.meanT_daily + (da.tile(self.elevation[:,:,np.newaxis],(1,1,self.doy_end)).rechunk(chunks=self.chunk3D)/100*0.55)
    #     self.P_by_PET_daily = da.nan_to_num(precipitation / self.pet_daily)
    # else:
    #     self.meanT_daily_sealevel = self.meanT_daily + np.expand_dims(self.elevation/100*0.55,axis=2) # automatic broadcasting #KLG        
    #     self.P_by_PET_daily = np.nan_to_num(precipitation / self.pet_daily)

    # self.set_monthly = False

    # print('computing interp_daily_temp')
    # # smoothed daily temperature
    # # create a mask of all 1's if the user doesn't provide a mask
    # if self.set_mask:
    #     mask=self.im_mask
    # else:
    #     mask=np.ones((self.im_height,self.im_width),dtype='int')

    # obj_utilities = UtilitiesCalc.UtilitiesCalc(self.chunk2D,self.chunk3D)  #KLG

    # # if self.parallel:
    # self.interp_daily_temp=obj_utilities.smoothDailyTemp(self.doy_start,self.doy_end, mask, self.meanT_daily,self.chunk3D)
    # # else:
    #     # self.interp_daily_temp=obj_utilities.smoothDailyTemp(self.doy_start,self.doy_end, mask, self.meanT_daily)  

    # print('copying maxT_daily,minT_daily,totalPrec_daily')
    # self.maxT_daily = max_temp
    # # self.minT_daily = min_temp
    # self.totalPrec_daily = precipitation  #KLG    
    
          


In [ ]:
setDailyClimateData(clim_reg, min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

In [ ]:
clim_reg.pet_daily.shape,clim_reg.nchunks

In [ ]:
data_labels=clim_reg.__dict__.keys()
data_labels

In [ ]:
if clim_reg.set_mask:
    mask=clim_reg.im_mask
else:
    mask=np.ones((clim_reg.im_height,clim_reg.im_width),dtype='int')

# this takes 100s
# days = np.arange(clim_reg.doy_start,clim_reg.doy_end+1) # x values  #KLG
# mask3D = np.tile(mask[:,:,np.newaxis], (1,1,days.shape[0]))  #KLG
# data=np.where(mask3D==0,0,clim_reg.meanT_daily)   #KLG
# data2D=data.transpose(2,0,1).reshape(days.shape[0],-1) # every column is a set of y values  #KLG
# del data  #KLG
# quad_spl=np.polynomial.polynomial.polyfit(days,data2D,deg=5)  #KLG
# interp_daily_temp=np.polynomial.polynomial.polyval(days,quad_spl)  #KLG
# interp_daily_temp=interp_daily_temp.reshape(mask3D.shape[0],mask3D.shape[1],-1)  #KLG
# interp_daily_temp=np.where(mask3D==0,np.nan,interp_daily_temp)        
# interp_daily_temp.shape

# this takes 33s
# days = np.arange(clim_reg.doy_start,clim_reg.doy_end+1) # x values  #KLG
# mask3D = da.tile(mask[:,:,np.newaxis], (1,1,days.shape[0])).rechunk(chunks=clim_reg.chunk3D)  #KLG
# data=da.where(mask3D==0,0,clim_reg.meanT_daily)   #KLG
# nlats=data.shape[0]
# data2D=data.transpose(2,0,1).reshape(days.shape[0],-1).rechunk(chunks=(clim_reg.chunk3D[0],clim_reg.chunk3D[1]*nlats)) # every column is a set of y values  #KLG
# coefs=np.polynomial.polynomial.polyfit(days,data2D,deg=5)  #KLG
# interp_daily_temp=np.polynomial.polynomial.polyval(days,coefs)  #KLG
# interp_daily_temp.shape


# # this takes 33s
# days = np.arange(clim_reg.doy_start,clim_reg.doy_end+1) # x values  #KLG
# mask3D = da.tile(mask[:,:,np.newaxis], (1,1,days.shape[0])).rechunk(chunks=clim_reg.chunk3D)  #KLG
# data=da.where(mask3D==0,0,clim_reg.meanT_daily)   #KLG
# nlats=data.shape[0]
# data2D=data.transpose(2,0,1).reshape(days.shape[0],-1).rechunk(chunks=(clim_reg.chunk3D[0],clim_reg.chunk3D[1]*nlats))#(-1,-1)) # every column is a set of y values  #KLG
# def polyfit(x,y,deg):
#     coefs=np.polynomial.polynomial.polyfit(x,y,deg=deg)
#     return coefs
# deg=5
# coefs=da.apply_gufunc(polyfit,"(t),(t,s),()->(d,s)",days,data2D,deg,output_dtypes='float32',output_sizes={'t':data2D.shape[0],'s':data2D.shape[1],'d':deg+1},vectorize=True)#.compute()
# def polyval(x,coefs):
#     spline=np.polynomial.polynomial.polyval(x,coefs)    
#     return spline
# interp_daily_temp=da.apply_gufunc(polyval,"(t),(d,s)->(s,t)",days,coefs,output_dtypes='float32',output_sizes={'t':data2D.shape[0],'s':data2D.shape[1],'d':deg+1},vectorize=True).compute()
# interp_daily_temp



In [ ]:
if clim_reg.set_mask:
    mask=clim_reg.im_mask
else:
    mask=np.ones((clim_reg.im_height,clim_reg.im_width),dtype='int')

days1D = np.arange(clim_reg.doy_start,clim_reg.doy_end+1) # x values  #KLG
mask3D = da.tile(mask[:,:,np.newaxis], (1,1,days1D.shape[0])).rechunk(chunks=clim_reg.chunk3D)  #KLG
data=da.where(mask3D==0,0,clim_reg.meanT_daily)   #KLG
nlats=data.shape[0]
data2D=data.transpose(2,0,1).reshape(days1D.shape[0],-1).rechunk({0:-1,1:'auto'})

days1D=dask.delayed(days1D)
partitions=data2D.to_delayed()

def polyfit_polyval(x,y,deg):
    coefs=np.polynomial.polynomial.polyfit(x,y,deg=deg)         
    spline=np.polynomial.polynomial.polyval(x,coefs) 
    return spline   

deg=5
delayed_values = [dask.delayed(polyfit_polyval)(days1D,part,deg) for part in partitions[0,:]]
results=dask.compute(*delayed_values)

interp_daily_temp=np.concatenate(results).reshape(mask3D.shape[0],mask3D.shape[1],-1)  #KLG
# interp_daily_temp.shape


In [ ]:
data2D

In [ ]:
# results[0].shape
test=np.concatenate(results)
test.shape

In [ ]:
if clim_reg.set_mask:
    mask=clim_reg.im_mask
else:
    mask=np.ones((clim_reg.im_height,clim_reg.im_width),dtype='int')

days1D = np.arange(clim_reg.doy_start,clim_reg.doy_end+1) # x values  #KLG
# days1D = da.arange(clim_reg.doy_start,clim_reg.doy_end+1,chunks=-1) # x values  #KLG
mask3D = np.tile(mask[:,:,np.newaxis], (1,1,days1D.shape[0])) #KLG
data=np.where(mask3D==0,0,clim_reg.meanT_daily)   #KLG
data2D=data.transpose(2,0,1).reshape(days1D.shape[0],-1)#.rechunk(chunks=-1)#(clim_reg.chunk3D[0],clim_reg.chunk3D[1]*nlats))#(-1,-1)) # every column is a set of y values  #KLG
ngrids=data2D.shape[1]
# days2D = da.from_array(np.tile(days1D[:,np.newaxis],(1,data2D.shape[1])),chunks=(clim_reg.chunk3D[0],clim_reg.chunk3D[1]*nlats)) # x values  #KLG

# @dask.delayed
# def polyfit(x,y,deg):
#     coefs=np.polynomial.polynomial.polyfit(x,y,deg=deg)
#     return coefs

# def polyval(x,coefs):
#     spline=np.polynomial.polynomial.polyval(x,coefs)    
#     return spline   
# days1D=dask.persist(days1D)
# data2D=dask.persist(data2D)
days1D=dask.delayed(days1D)
data2D=dask.delayed(data2D)

@dask.delayed
def polyfit_polyval(x,y,deg):
    coefs=np.polynomial.polynomial.polyfit(x,y,deg=deg)         
    spline=np.polynomial.polynomial.polyval(x,coefs) 
    return spline   

deg=5
nlats_block=int(np.ceil(ngrids/clim_reg.nchunks))
# ngrids,clim_reg.nchunks,nlats_block

# interp_T=np.empty((data2D.shape[1],data2D.shape[0]),dtype='float32')

task_list=[]
for ichunk in range(clim_reg.nchunks)[0:3]:
    istart=ichunk*nlats_block
    if ichunk < clim_reg.nchunks-1:
        iend=istart+nlats_block
    else:
        iend=ngrids
    print(istart,iend)
    interp_task=polyfit_polyval(days1D,data2D[:,istart:iend],deg)
    task_list.append(interp_task)
print('computing')
interp_list=dask.compute(*task_list)

interp_list[0].shape
# interp_temp_daily=np.concat(interp_list,axis=1)

# task_list=[]
# for ichunk in range(clim_reg.nchunks)[0:3]:
#     istart=ichunk*nlats_block
#     if ichunk < clim_reg.nchunks-1:
#         iend=istart+nlats_block
#     else:
#         iend=ngrids

#     interp_task=polyval(days1D,coefs_list[ichunk])
#     task_list.append(coefs_task)
# coefs_list=dask.compute(*task_list)

#     print(coefs.T)

        # T=polyval(days1D,coefs)

    # print(istart,iend)
# days1D.shape,data2D.shape

# for block in data2D.blocks:
#     coef_block=polyfit(days1D,block)

# data2D.blocks[0,1]

# coefs=da.apply_gufunc(polyfit,"(t),(t,s),()->(d,s)",days,data2D,deg,output_dtypes='float32',output_sizes={'t':data2D.shape[0],'s':data2D.shape[1],'d':deg+1},vectorize=True)#.compute()
# coefs=da.map_blocks(polyfit,days2D,data2D,dtype='float32',chunks=(deg+1,clim_reg.chunk3D[1]*nlats),drop_axis=0,new_axis=0,meta=np.empty((6,data2D.shape[1]),dtype='float32'))#.compute()
# coefs.shape
# def polyval(x,coefs):
#     spline=np.polynomial.polynomial.polyval(x,coefs)    
#     return spline
# # interp_daily_temp=da.apply_gufunc(polyval,"(t),(d,s)->(s,t)",days,coefs,output_dtypes='float32',output_sizes={'t':data2D.shape[0],'s':data2D.shape[1],'d':deg+1},vectorize=True).compute()
# interp_daily_temp=da.apply_gufunc(polyval,"(t),(d,s)->(s,t)",days1D,coefs,output_dtypes='float32',output_sizes={'t':data2D.shape[0],'s':data2D.shape[1],'d':deg+1},vectorize=True,allow_rechunk=True).compute()
# interp_daily_temp

In [ ]:
data2D=min_temp.transpose(2,0,1).reshape(365,-1).rechunk(chunks=(-1,7200)) # every column is a set of y values  #KLG
data2D

In [ ]:
clim_reg.__dict__.keys()

In [ ]:
clim_reg.pet_daily.dtype

In [ ]:
    def setDailyClimateData(self, min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity):
        """Load DAILY climate data into the Class and calculate the Reference Evapotranspiration (ETo)

        Args:
            min_temp (3D NumPy): Daily minimum temperature [Celcius]
            max_temp (3D NumPy): Daily maximum temperature [Celcius]
            precipitation (3D NumPy): Daily total precipitation [mm/day]
            short_rad (3D NumPy): Daily solar radiation [W/m2]
            wind_speed (3D NumPy): Daily windspeed at 2m altitude [m/s]
            rel_humidity (3D NumPy): Daily relative humidity [percentage decimal, 0-1]
        """
        self.doy_start=1  #KLG
        self.doy_end=min_temp.shape[2]  #KLG

        rel_humidity[rel_humidity > 0.99] = 0.99
        rel_humidity[rel_humidity < 0.05] = 0.05
        short_rad[short_rad < 0] = 0
        wind_speed[wind_speed < 0] = 0
                
        # self.meanT_daily = np.zeros((self.im_height, self.im_width, 365)) 
        # self.totalPrec_daily = np.zeros((self.im_height, self.im_width, 365)) 
        # self.pet_daily = np.zeros((self.im_height, self.im_width, 365))  
        self.maxT_daily = max_temp
        self.minT_daily = min_temp
        self.totalPrec_daily = precipitation  #KLG
        del max_temp, min_temp, precipitation  #KLG


        # calculation of reference evapotranspiration (ETo)  #KLG
        obj_eto = ETOCalc.ETOCalc(self.doy_start, self.doy_end, self.latitude, self.elevation)  #KLG
        shortrad_daily_MJm2day = (short_rad*3600.*24.)/1000000. # convert w/m2 to MJ/m2/day  #KLG
        obj_eto.setClimateData(self.minT_daily, self.maxT_daily, wind_speed, shortrad_daily_MJm2day, rel_humidity)  #KLG
        del rel_humidity,short_rad,wind_speed,shortrad_daily_MJm2day  #KLG
        self.pet_daily= obj_eto.calculateETO()   #KLG
        del obj_eto

        # sea level temperature
        # self.meanT_daily_sealevel = self.meanT_daily + np.tile(np.reshape(self.elevation/100*0.55, (self.im_height,self.im_width,1)), (1,1,365))
        self.meanT_daily = 0.5*(self.minT_daily+self.maxT_daily)  #KLG
        self.meanT_daily_sealevel = self.meanT_daily + np.expand_dims(self.elevation/100*0.55,axis=2) # automatic broadcasting #KLG        

        # P over PET ratio (to eliminate nan in the result, nan is replaced with zero)
        self.P_by_PET_daily = np.nan_to_num(self.totalPrec_daily / self.pet_daily)
        self.set_monthly = False

        # smoothed daily temperature
        obj_utilities = UtilitiesCalc.UtilitiesCalc()  #KLG

        if self.set_mask:
            mask=self.im_mask
        else:
            mask=np.ones((self.im_height,self.im_width),dtype='int')

        if self.parallel:
            self.interp_daily_temp=obj_utilities.smoothDailyTemp(self.doy_start,self.doy_end, mask, self.meanT_daily,self.chunk3D)
        else:
            self.interp_daily_temp=obj_utilities.smoothDailyTemp(self.doy_start,self.doy_end, mask, self.meanT_daily)


        self.chunk3D=(-1,94,-1)
        self.chunk2D=(-1,94)

In [ ]:
# # expect ~30 seconds run time for china

# start=timer()

# if daily:
#     clim_reg.setDailyClimateData(
#         min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
# else:
#     clim_reg.setMonthlyClimateData(
#         min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)
    
# task_time=timer()-start
# task_time    

In [ ]:
'Releasing the memory of input climate data -- free up some RAM space'
# del(min_temp, max_temp, precipitation, short_rad, wind_speed, rel_humidity)

In [ ]:
# get a list of all the data in the class instance clim_reg

data_labels=clim_reg.__dict__.keys()
data_labels

In [ ]:
# # sanity check
# # plot 2D & 3D data, print value of scalars 

# for key in data_labels:
#     print(key)    
#     try:
#         ndims=len(clim_reg.__getattribute__(key).shape)
#         # dims=clim_reg.__getattribute__(key).shape
#     except:
#         ndims=1
#     if ndims==1:
#         print(key,'=',clim_reg.__getattribute__(key))
#     if ndims==2:
#         plt.imshow(clim_reg.__getattribute__(key),interpolation='none')
#         plt.colorbar(shrink=0.5)
#         plt.title(key)
#         plt.show()
#     if ndims==3:
#         plt.imshow(clim_reg.__getattribute__(key)[:,:,0],interpolation='none')
#         plt.colorbar(shrink=0.5)
#         plt.title(key)
#         plt.show()

In [ ]:
# data_list=[]
# for key in data_labels:
#     data_list.append(clim_reg.__getattribute__(key))

# data_list

In [ ]:
# fut_climreg=dask.persist(*data_list)

# fut_climreg.result()

In [ ]:
# da_vars=clim_reg.meanT_daily
# da_vars.store(clim_reg.meanT_daily,return_stored=True)
# da_vars


In [ ]:
# fut_climreg

NOTE: After loading the data into the *clim_reg* Class, all the parameters will be converted to DAILY DATA and calculated as other parameters. 
These new parameters are available and can be called into use as:
- *clim_reg.minT_daily* (minimum temperature)
- *clim_reg.maxT_daily* (maximum temperature)
- *clim_reg.meanT_daily* (mean temperature)
- *clim_reg.meanT_daily_sealevel* (mean temperature, corrected to sea-level)
- *clim_reg.totalPrec_daily* (total precipitation)
- *clim_reg.pet_daily* (reference evapotranspiration, ETo)
- *clim_reg.P_by_PET_daily* (ratio of precipitation over ETo)
  


#### Thermal Climate
The Thermal Climate function calculates and classifies latitudinal thermal climate, which will be used later in Module 2 for the assessment of potential crops and land utilization types (LUT) presence in each grid cell.

In [ ]:
# expect >1s run time for china
# non-vectorized: expect ~50 seconds run time for china
start=timer()

tclimate = clim_reg.getThermalClimate()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(tclimate, cmap=plt.get_cmap('gist_ncar_r', 12),vmin=-0.3,vmax=12.5,interpolation='none')
plt.title('Thermal Climate')
plt.colorbar(shrink=0.4)
plt.savefig(out_path+"ThermalClimate.png",bbox_inches ="tight",dpi=300) #Save as PNG image
plt.show()

In [ ]:
obj_utilities.saveRaster(mask_path, out_path+'ThermalClimate_'+revname+'.tif',tclimate) #Save as GeoTIFF raster

#### Thermal Zone
The thermal zone is classified based on actual temperature which reflects on the temperature regimes of major thermal climates

In [ ]:
# expect >1s run time for china
# non-vectorized: expect ~35 seconds run time for china
start=timer()

tzone = clim_reg.getThermalZone()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(tzone, cmap=plt.get_cmap('gist_ncar_r', 12),vmin=-0.3,vmax=12.5,interpolation='none')
plt.title('Thermal Zones')
plt.colorbar(shrink=0.4)
plt.savefig(out_path+"ThermalZone"+revname+".png",bbox_inches ="tight",dpi=300) #Save as PNG image
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'ThermalZone'+revname+'.tif',tzone) #Save as GeoTIFF raster

#### Thermal Length of Growing Period (LGP)
The thermal length of growing period (LGPt) is defined as the number of days in a year during which the daily mean temperature (Ta) is conductive to crop growth and development. PyAEZ utilizes the AEZ three standard temperature thresholds for LGPt:
- Periods with Ta>0°C (LGPt0)
- Periods with Ta>5°C (LGPt5) – the period conductive to plant growth and development
- Periods, and Ta>10°C (LGPt10) – a proxy for the period of low risks for late and early frost occurrences and termed ‘frost-free period’

In [ ]:
# expect >1s run time for china
# non-vectorized: expect ~1 second run time for china
start=timer()

lgpt0 = clim_reg.getThermalLGP0()
lgpt5 = clim_reg.getThermalLGP5()
lgpt10 = clim_reg.getThermalLGP10()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
#======================
# plt.figure(figsize=(20, 4),dpi=300)
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(lgpt0,vmin=0,vmax=366,interpolation='none')
plt.title('LGPt 0')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 2)
plt.imshow(lgpt5, vmin=0, vmax=366,interpolation='none')
plt.title('LGPt 5')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 3)
plt.imshow(lgpt10, vmin=0, vmax=366,interpolation='none')
plt.title('LGPt 10')
plt.colorbar(shrink=0.8)
#----------------------
plt.savefig(out_path+"ThermalLGPs"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
#======================

obj_utilities.saveRaster(mask_path, out_path+'LGPt0_'+revname+'.tif', lgpt0)
obj_utilities.saveRaster(mask_path, out_path+'LGPt5_'+revname+'.tif', lgpt5)
obj_utilities.saveRaster(mask_path, out_path+'LGPt10_'+revname+'.tif', lgpt10)


#### Temperature Sum

In [ ]:
# expect >1s run time for china
# non-vectorized: expect ~1 second run time for china
start=timer()

tsum0 = clim_reg.getTemperatureSum0()
tsum5 = clim_reg.getTemperatureSum5()
tsum10 = clim_reg.getTemperatureSum10()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
#======================
# plt.figure(1, figsize=(20, 4),dpi=300)
plt.figure()
plt.subplot(1, 3, 1)
plt.imshow(tsum0, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 0')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 2)
plt.imshow(tsum5, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 5')
plt.colorbar(shrink=0.8)
#----------------------
plt.subplot(1, 3, 3)
plt.imshow(tsum10, cmap='hot_r', vmin=0, vmax=11000,interpolation='none')
plt.title('T-sumation 10')
plt.colorbar(shrink=0.8)
#----------------------
plt.savefig(out_path+"Tsum_"+revname+".png",
            bbox_inches="tight", dpi=300)
plt.show()
#======================

In [ ]:
obj_utilities.saveRaster(mask_path, out_path+'tsum0_'+revname+'.tif', tsum0)
obj_utilities.saveRaster(mask_path, out_path+'tsum5_'+revname+'.tif', tsum5)
obj_utilities.saveRaster(mask_path, out_path+'tsum10_'+revname+'.tif', tsum10)

#### Temperature Profile

In [ ]:
# vectorized+parallel: expect ~1 second run time for china
# vectorized: expect ~6 seconds run time for china
# non-vectorized: expect ~50 seconds run time for china
start=timer()

tprofile = clim_reg.getTemperatureProfile()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
tile_list = ['A1','A2','A3','A4','A5','A6','A7','A8','A9',
            'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9']

# fig = plt.figure(figsize=(12, 12))
fig = plt.figure()
for i1 in range(1, 19):
    plt.subplot(6, 3, i1)
    plt.imshow(tprofile[i1-1],interpolation='none')
    plt.title(tile_list[i1-1])
    plt.colorbar(shrink=0.8)
plt.tight_layout()

plt.savefig(out_path+"Tprofiles_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()


In [ ]:
for i1 in range(18):
    obj_utilities.saveRaster(mask_path, out_path+'TProfile_' + tile_list[i1] +'_'+revname+'.tif', tprofile[i1])

#### Length of Growing Periods (LGPs)

In [ ]:
@dask.delayed
def test_func1(x,y,z,X,Y,Z,mask):
    x=x*2
    y=y+3
    z=z**2
    X=np.where(mask,x,X)
    Y=np.where(mask,y,Y)
    Z=np.where(mask,z,Z)
    return X,Y,Z

def test_func2(out1,out2,out3):
    out1=out1+10
    out2=out2+20
    out3=out3+30
    return out1,out2,out3

@dask.delayed
def test_combine(x1,y1,z1,x2,y2,z2,mask1,mask2):
    x=np.empty(x1.shape)
    y=np.empty(x1.shape)
    z=np.empty(x1.shape)
    x=np.where(mask1,x1,x)
    x=np.where(mask2,x2,x)
    y=np.where(mask1,y1,y)
    y=np.where(mask2,y2,y)
    z=np.where(mask1,z1,z)
    z=np.where(mask2,z2,z)        
    return x,y,z




ndays=5
x_arr=np.ones((ndays,2,2),dtype='float32')
y_arr=np.zeros((ndays,2,2),dtype='float32')
z_arr=y_arr-1

X=np.empty(x_arr.shape)
Y=np.empty(y_arr.shape)
Z=np.empty(z_arr.shape)
X[:],Y[:],Z[:]=np.nan,np.nan,np.nan

# [[1,1],[1,1]]
mask1=np.array([[[1,0],[1,1]],[[1,0],[1,1]],[[1,0],[1,1]],[[1,0],[1,1]],[[1,0],[1,1]]])
mask2=np.array([[[0,1],[0,0]],[[0,1],[0,0]],[[0,1],[0,0]],[[0,1],[0,0]],[[0,1],[0,0]]])

mask1.shape

In [ ]:
ncats=5
for i,cat in enumerate(np.arange(ncats)+1):
    print(i,cat)

In [ ]:
task_list=[]
for doy in range(ndays):
    # task=test_func1(x_arr,y_arr,z_arr,X,Y,Z,mask1)
    # task_list.append(task)
    X,Y,Z=test_func1(x_arr,y_arr,z_arr,X,Y,Z,mask1)
    # task=test_func1(x_arr,y_arr,z_arr,X,Y,Z,mask2)
    # task_list.append(task)
    # task=test_combine()
    # print(task_list)
    # output=dask.compute(*task_list)


In [ ]:
len(output)
output[1]

In [ ]:
task_list=[]
for doy in range(ndays):
    # out1,out2,out3=test_func1(x_arr,y_arr,z_arr)
    task=test_func1(x_arr,y_arr,z_arr)
    # task=test_func2(out1,out2,out3)
    # task_list.append(task)
    output=task.compute()
    x_arr=output[0]
    y_arr=output[1]
    z_arr=output[2]
    

In [ ]:
output=dask.compute(*task_list)

In [ ]:
x_arr

In [ ]:
output[0][0]

In [ ]:
# vectorized+parallel: expect ~66 seconds run time for china
# vectorized: expect ~1.5 minutes run time for china
# non-vectorized: expect ~6 minutes run time for china
start=timer()

lgp = clim_reg.getLGP(Sa=100., D=1.)
lgp_class = clim_reg.getLGPClassified(lgp)
lgp_equv = clim_reg.getLGPEquivalent()

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
# fig = plt.figure(figsize=(20, 4))
fig = plt.figure()
# fig = plt.figure(figsize=(10, 8),dpi=600)
plt.subplot(121)
plt.imshow(lgp, cmap='viridis', vmin=0, vmax=366,interpolation='none')
plt.title('LGP [days]')
plt.colorbar(shrink=0.8)
# plt.savefig(out_path+"LGP_"+revname+".png", bbox_inches="tight", dpi=600)
# plt.show()

# fig = plt.figure(figsize=(20, 8),dpi=600)
plt.subplot(122)
plt.imshow(lgp_equv, cmap='viridis', vmin=0, vmax=366,interpolation='none')
plt.title('LGP Equivalent [days]')
plt.colorbar(shrink=0.8)
# plt.savefig(out_path+"LGP_Equv_"+revname+".png",bbox_inches="tight", dpi=600)
plt.show()

plt.savefig(out_path+"LGP_and_LGP_Equv_"+revname+".png",bbox_inches="tight", dpi=300)


obj_utilities.saveRaster(mask_path, out_path+'LGP_'+revname+'.tif', lgp)
obj_utilities.saveRaster(mask_path, out_path+'LGPEquivalent_'+revname+'.tif', lgp_equv)


#### Multi Cropping Zone
Multiple cropping zones classification is an additional agro-climatic indicator, which relates to the possibility of cultivating multiple sequential crops under rain-fed and irrigated conditions.

In [ ]:
# expect >1s run time for china
# non-vectorized: expect ~1.5 minutes run time for china
start=timer()

multi_crop = clim_reg.getMultiCroppingZones(tclimate, lgp, lgpt5, lgpt10, tsum0, tsum10)
multi_crop_rainfed = multi_crop[0]  # for rainfed conditions
multi_crop_irr = multi_crop[1]  # for irrigated conditions

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(multi_crop_irr, cmap=plt.get_cmap('gist_ncar_r', 9), vmin=-0.2, vmax=8.4,interpolation='none')
plt.title('Multi Cropping Zone - IRR')
plt.colorbar()
plt.savefig(out_path+"multicrop_irr_"+revname+".png", bbox_inches="tight", dpi=300)
plt.show()

fig = plt.figure()
plt.imshow(multi_crop_rainfed,cmap=plt.get_cmap('gist_ncar_r', 9), vmin=-0.2, vmax=8.4,interpolation='none')
plt.title('Multi Cropping Zone - RAINFED')
plt.colorbar()
plt.savefig(out_path+"multicrop_rain_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'multicrop_irr_'+revname+'.tif', multi_crop_irr)
obj_utilities.saveRaster(mask_path, out_path+'multicrop_rain_'+revname+'.tif', multi_crop_rainfed)


### Air Frost Index and Permafrost Evaluation
Occurrence of continuous or discontinuous permafrost conditions are used in the suitability assessment. Permafrost areas are characterized by sub-soil at or below the freezing point for two or more years. In this section, PyAEZ utilizes the air frost index (FI) which is used to characterize climate-derived permafrost condition into 4 classes: 
1) Continuous permafrost
2) Discontinuous permafrost 
3) Sporadic permafrost
4) No permafrost

In [ ]:
# expect >1s run time for china
# non-vectorized: expect ~2 seconds run time for china
start=timer()

permafrost_eval = clim_reg.AirFrostIndexandPermafrostEvaluation()
frost_index = permafrost_eval[0]
permafrost = permafrost_eval[1]

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(frost_index, cmap=plt.get_cmap('tab20b', 11), vmin=-0.05, vmax=1.05,interpolation='none')
plt.title('Frost Index')
plt.colorbar()
plt.savefig(out_path+"frost_index_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

fig = plt.figure()
plt.imshow(permafrost, cmap=plt.get_cmap('tab20b', 5), vmin=-0.5, vmax=4.3,interpolation='none')
plt.title('Permafrost Evaluation')
plt.colorbar()
plt.savefig(out_path+"permafrost_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()


obj_utilities.saveRaster(mask_path, out_path+'frost_index_'+revname+'.tif', frost_index)
obj_utilities.saveRaster(mask_path, out_path+'permafrost_'+revname+'.tif', permafrost)


### Fallow period requirement
Fallow is an agricultural technique that consists of not sowing the arable land during one or more growing seasons. In AEZ framework, the fallow factors have been established by main crop groups and environmental conditions. The crop groups include cereals, legumes, roots and tubers, and a miscellaneous group consisting of long-term annuals/perennials. The fallow factors are expressed as percentage of time during the fallow-cropping cycle the land must be under fallow. PyAEZ determines the fallow requirements using Thermal Zones.

In [ ]:
# expect >1s run time for china
# expect ~15 seconds run time for china
start=timer()

tzone_fallow = clim_reg.TZoneFallowRequirement(tzone)

task_time=timer()-start
task_time

In [ ]:
'''save and visualize result'''
fig = plt.figure()
plt.imshow(tzone_fallow, cmap=plt.get_cmap('tab10', 7), vmin=-0.5, vmax=6.3,interpolation='none')
plt.title('Fallow Requirement')
plt.colorbar()
plt.savefig(out_path+"fallow_"+revname+".png",bbox_inches="tight", dpi=300)
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'fallow_'+revname+'.tif', tzone_fallow)


In [ ]:
# check dtypes of all outputs
outputs=[tclimate,tzone,lgpt0,lgpt5,lgpt10,tsum0,tsum5,tsum10,tprofile,lgp,lgp_class,lgp_equv,multi_crop_rainfed,multi_crop_irr,frost_index,permafrost,tzone_fallow]
dtypes=[]
for var in outputs:
    # for arrays
    try:
        dt=var.dtype
        dtypes.append(dt)
    # for lists of arrays
    except:
        for v in var:
            dt=v.dtype
            dtypes.append(dt)

print(dtypes)

### Agro-ecological zones classification
The agro-ecological zones (AEZ) methodology provides a framework for establishing a spatial inventory of land resources compiled from global/national environmental data sets and assembled to quantify multiple spatial characteristics required for the assessments of land productivity under location-specific agro-ecological conditions.

In [ ]:
# can't run until we have the global soil_terrain_lulc

# expect ~ minutes run time for global
start=timer()

aez = clim_reg.AEZClassification(
    tclimate, lgp, lgp_equv, lgpt5, soil_terrain_lulc, permafrost)

task_time=timer()-start
task_time

In [ ]:
# now visualizing result
fig = plt.figure(dpi=600)
plt.imshow(aez, cmap=plt.get_cmap('rainbow', 59), vmin=0, vmax=59,interpolation=None)
plt.title('Agro-ecological Zonation')
plt.colorbar()
plt.savefig(out_path+"aez_"+revname+".png",bbox_inches="tight", dpi=600)
plt.show()

obj_utilities.saveRaster(mask_path, out_path+'aez_'+revname+'.tif', aez)


<hr>

### END OF MODULE 1: CLIMATE REGIME

<hr>